In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
os.chdir("/content/drive/My Drive/FCDO")
os.getcwd()

In [1]:
import os
import csv
import json
import re
import numpy as np
import pandas as pd
from unicodedata import normalize 

In [2]:
import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [3]:
from pdfminer.high_level import extract_text

In [4]:
# replace latin characters. é -> e , et al.
def simplify(text):
    import unicodedata
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

# Step1: link pdfs to Indigo dataset

### Load article lists from Indigo dataset

In [5]:
input_file_csv1 = 'Data/Articles lists/review_128867_included_csv_20210828012212.csv'
input_file_csv2 = 'Data/Articles lists/review_128867_select_csv_20210828011947.csv'

pdf_files_folder = 'Data/Pdfs/Full text PDF'
# pdf_files_folder = 'Data/Pdfs/all_pdfs'
output_file_json = 'step1_output.json'

In [6]:
texts_df1 = pd.read_csv(input_file_csv1)  
texts_df2 = pd.read_csv(input_file_csv2)  

In [7]:
texts_df = pd.concat([texts_df1, texts_df2])
texts_df['Study'] = texts_df['Study'].replace(np.nan, '').apply(simplify).apply(lambda x: x.strip())
texts_df['Title'] = texts_df['Title'].replace(np.nan, '').apply(simplify).apply(lambda x: x.strip())
texts_df['Covidence #'] = texts_df['Covidence #'].replace(np.nan, '').apply(simplify).apply(lambda x: x.strip())

In [8]:
print(len(texts_df[texts_df['Covidence #'] != '']))

1952


### List PDFs including id number (#12345) and extract filename information

In [9]:
file_names = []
file_numbers = []
file_authors = []
file_years = []

for f in os.listdir(pdf_files_folder):
    if os.path.isfile(os.path.join(pdf_files_folder, f)):
        if re.search('#\d*',f):
            file_names.append(f)            
            file_numbers.append(re.findall('#[0-9]*', f)[0].strip())
            temp_auth = re.findall('[^ 0-9#][ a-zA-Z()-]*', f)[0].strip("-").strip()
            if temp_auth != '.pdf':
                file_authors.append(temp_auth)
            else:
                file_authors.append('')
            temp_year = re.findall('[^0-9#][0-9]{4}', f, re.IGNORECASE)
            if len(temp_year) > 0:
                temp_year = temp_year[0].strip()
                file_years.append(temp_year)        
            else:
                file_years.append('')
            print(os.path.join(pdf_files_folder,f))
            
pdf_files_df = pd.DataFrame()
pdf_files_df['filename'] = file_names
pdf_files_df['number'] = file_numbers
pdf_files_df['author'] = file_authors
pdf_files_df['year'] = file_years

Data/Pdfs/Full text PDF/#8561 - Skopec 2018.pdf
Data/Pdfs/Full text PDF/#6580 - Sherry 2012.pdf
Data/Pdfs/Full text PDF/#5545 - Basinga 2010.pdf
Data/Pdfs/Full text PDF/#15791 - Acevedo 2016.pdf
Data/Pdfs/Full text PDF/#2065 - Storeng 2014.pdf
Data/Pdfs/Full text PDF/#15950 - Castaneda-Hernandez 2012.pdf
Data/Pdfs/Full text PDF/#17442 - JaninePlaisier 2018.pdf
Data/Pdfs/Full text PDF/#2579 - Dopp 2019.pdf
Data/Pdfs/Full text PDF/#7530 - Silow-Carroll 2008.pdf
Data/Pdfs/Full text PDF/#8599 - Bieretz 2019.pdf
Data/Pdfs/Full text PDF/#4261 - Arena 2016.pdf
Data/Pdfs/Full text PDF/#7105 - Young 2010.pdf
Data/Pdfs/Full text PDF/#17057 - Borowitz 2014.pdf
Data/Pdfs/Full text PDF/#8491 - Sanderstrom 2016.pdf
Data/Pdfs/Full text PDF/#17102 - LucLaviolette 2016.pdf
Data/Pdfs/Full text PDF/#6802 - Lahti 1998.pdf
Data/Pdfs/Full text PDF/#3875 - Barry 2012.pdf
Data/Pdfs/Full text PDF/#13630 - Flynn 1997.pdf
Data/Pdfs/Full text PDF/#1858 - Peckham 2008.pdf
Data/Pdfs/Full text PDF/#11109 - Wilhelm 2

Data/Pdfs/Full text PDF/#5885 - Fralick 2020.pdf
Data/Pdfs/Full text PDF/#8546 - Hawkins 2017.pdf
Data/Pdfs/Full text PDF/#16386 - Lehmann 2018.pdf
Data/Pdfs/Full text PDF/#7176 - Lee 2014.pdf
Data/Pdfs/Full text PDF/#12250 - Chiroleu-Assouline 2012.pdf
Data/Pdfs/Full text PDF/#17305.pdf
Data/Pdfs/Full text PDF/#8458 - Francis 2015.pdf
Data/Pdfs/Full text PDF/#17122 - NeilStanworth 2020.pdf
Data/Pdfs/Full text PDF/#15985 - Lage 2012.pdf
Data/Pdfs/Full text PDF/#5704 - Manitu 2015.pdf
Data/Pdfs/Full text PDF/#4112 - Streib 1995.pdf
Data/Pdfs/Full text PDF/Koning and Heinrich, 2013 #1917.pdf
Data/Pdfs/Full text PDF/#7354 - Fleischman 2017.pdf
Data/Pdfs/Full text PDF/#15688 - Rosenthal 2007.pdf
Data/Pdfs/Full text PDF/#15427 - Dickson 2014.pdf
Data/Pdfs/Full text PDF/#10389 - Diop 2017.pdf
Data/Pdfs/Full text PDF/#9132 - Lattemann 2009.pdf
Data/Pdfs/Full text PDF/#17251 - Clive 2011.pdf
Data/Pdfs/Full text PDF/#17846 - BigIssueInvestFundManagementLtd(BIIFM) 2020.pdf
Data/Pdfs/Full text PD

Data/Pdfs/Full text PDF/#17244 - Coles 2019.pdf
Data/Pdfs/Full text PDF/#8492 - Gelb 2016.pdf
Data/Pdfs/Full text PDF/#9460 - Kang 2019.pdf
Data/Pdfs/Full text PDF/#4344 - Shortell 2002.pdf
Data/Pdfs/Full text PDF/#17136 - IngoBräuer 2006.pdf
Data/Pdfs/Full text PDF/#17828 - Communities 2016.pdf
Data/Pdfs/Full text PDF/#1318 - Boyle 1998.pdf
Data/Pdfs/Full text PDF/McDonald et al., 2008 #13310.pdf
Data/Pdfs/Full text PDF/#16309 - Gray 2018.pdf
Data/Pdfs/Full text PDF/#1723 - Zhou 2012.pdf
Data/Pdfs/Full text PDF/#8177 - Vinson 1999.pdf
Data/Pdfs/Full text PDF/#5744 - Thornton 2016.pdf
Data/Pdfs/Full text PDF/#17119 - Diaconu 2020.pdf
Data/Pdfs/Full text PDF/#8612 - Gustafsson-Wright 2020.pdf
Data/Pdfs/Full text PDF/#17177 - Consulting 2014.pdf
Data/Pdfs/Full text PDF/#8398.pdf
Data/Pdfs/Full text PDF/#6911 - Ryan 2008.pdf
Data/Pdfs/Full text PDF/#17277 - Roder 2020.pdf
Data/Pdfs/Full text PDF/#2497 - Gosling 2018.pdf
Data/Pdfs/Full text PDF/#11019 - Öhman 2015.pdf
Data/Pdfs/Full text

Data/Pdfs/Full text PDF/#1625 - Stewart 2010.pdf
Data/Pdfs/Full text PDF/#5237 - Arling 2009.pdf
Data/Pdfs/Full text PDF/#5058 - Hubel 2013.pdf
Data/Pdfs/Full text PDF/#12361 - Ferman 2004.pdf
Data/Pdfs/Full text PDF/#17203 - EmilySusannahGraceHulse 2021.pdf
Data/Pdfs/Full text PDF/#16745 - Urick 2019.pdf
Data/Pdfs/Full text PDF/#17164.pdf
Data/Pdfs/Full text PDF/#17755 - Ecorys 2019.pdf
Data/Pdfs/Full text PDF/#6346 - Mabli 2017.pdf
Data/Pdfs/Full text PDF/#17716 - L 2021.pdf
Data/Pdfs/Full text PDF/#8551 - Hayford 2017.pdf
Data/Pdfs/Full text PDF/#2546 - Dansereau 2019.pdf
Data/Pdfs/Full text PDF/#2371 - Hillman 2017.pdf
Data/Pdfs/Full text PDF/#17391 - Langpap 2015.pdf
Data/Pdfs/Full text PDF/#8592 - Bieretz 2019.pdf
Data/Pdfs/Full text PDF/#16214 - deMariz 2018.pdf
Data/Pdfs/Full text PDF/#17726 - JeraldineKay 2020.pdf
Data/Pdfs/Full text PDF/#2488 - Li 2018.pdf
Data/Pdfs/Full text PDF/#17325 - Denmark 2016.pdf
Data/Pdfs/Full text PDF/#7086 - James 2009.pdf
Data/Pdfs/Full text PDF/

Data/Pdfs/Full text PDF/#1941 - Jackson 2013.pdf
Data/Pdfs/Full text PDF/#6416 - Fox 2011.pdf
Data/Pdfs/Full text PDF/#1567 - Faith 2010.pdf
Data/Pdfs/Full text PDF/#11703 - Lazzarini 2020.pdf
Data/Pdfs/Full text PDF/Schiller et al., 2000 #10837.pdf
Data/Pdfs/Full text PDF/#17777 - Management 2017.pdf
Data/Pdfs/Full text PDF/#17302 - Floyd 2017.pdf
Data/Pdfs/Full text PDF/#4230 - Malli 2014.pdf
Data/Pdfs/Full text PDF/#6470 - Reynolds 1993.pdf
Data/Pdfs/Full text PDF/#17188 - Nieveler 2017.pdf
Data/Pdfs/Full text PDF/#2266 - Oomkens 2016.pdf
Data/Pdfs/Full text PDF/#2480 - Meng 2018.pdf
Data/Pdfs/Full text PDF/#10254 - Herbst 2018.pdf
Data/Pdfs/Full text PDF/#17342 - Maier 2017.pdf
Data/Pdfs/Full text PDF/#5759 - Hendra 2016.pdf
Data/Pdfs/Full text PDF/#11734 - Sato 2020.pdf
Data/Pdfs/Full text PDF/#16204 - Sheather-Reid 2018.pdf
Data/Pdfs/Full text PDF/#6751 - Meacock 2017.pdf
Data/Pdfs/Full text PDF/#6663 - Wu 2015.pdf
Data/Pdfs/Full text PDF/#10922 - Reed 2014.pdf
Data/Pdfs/Full tex

Data/Pdfs/Full text PDF/Moore 1994 #5410.pdf
Data/Pdfs/Full text PDF/Anderson et al., 2015 #6278.pdf
Data/Pdfs/Full text PDF/#5768 - Gerrish 2017.pdf
Data/Pdfs/Full text PDF/#17415 - Fund 2018.pdf
Data/Pdfs/Full text PDF/#17049 - JonChristianson 2007.pdf
Data/Pdfs/Full text PDF/#2041 - Morgan 2014.pdf
Data/Pdfs/Full text PDF/#17759 - Consultants 2020.pdf
Data/Pdfs/Full text PDF/#3472 - Hossain 2012.pdf
Data/Pdfs/Full text PDF/#14389 - Mullen 2010.pdf
Data/Pdfs/Full text PDF/#17399 - Bank 2019.pdf
Data/Pdfs/Full text PDF/#16738 - Appel 2017.pdf
Data/Pdfs/Full text PDF/Copiello, 2016 #8866.pdf
Data/Pdfs/Full text PDF/#7661 - Kenny 2013.pdf
Data/Pdfs/Full text PDF/#17679.pdf
Data/Pdfs/Full text PDF/#17157 - JonKlinepeter 2019.pdf
Data/Pdfs/Full text PDF/#2599 - Hodgkin 2020.pdf
Data/Pdfs/Full text PDF/#17247 - Brookings 2017.pdf
Data/Pdfs/Full text PDF/#3674 - Hood 2002.pdf
Data/Pdfs/Full text PDF/#11418 - Wezel 2018.pdf
Data/Pdfs/Full text PDF/Garstka 2012 #4537.pdf
Data/Pdfs/Full text P

Data/Pdfs/Full text PDF/#17195 - Education 2019.pdf
Data/Pdfs/Full text PDF/#17772 - Sturgess 2017.pdf
Data/Pdfs/Full text PDF/#17190 - Infrastructure 2017.pdf
Data/Pdfs/Full text PDF/#5884 - Albertson 2020.pdf
Data/Pdfs/Full text PDF/#7729 - Peet 2015.pdf
Data/Pdfs/Full text PDF/#17699 - Program 2014.pdf
Data/Pdfs/Full text PDF/#8214 - Moon 2019.pdf
Data/Pdfs/Full text PDF/#17673 - Calleja 2015.pdf
Data/Pdfs/Full text PDF/#8318 - Sequist 2008.pdf
Data/Pdfs/Full text PDF/#2137 - Levy 2012.pdf
Data/Pdfs/Full text PDF/#17750 - Communities 2017.pdf
Data/Pdfs/Full text PDF/#2552 - Girth 2019.pdf
Data/Pdfs/Full text PDF/#17731 - International 2019.pdf
Data/Pdfs/Full text PDF/#6225 - Monaghan 2013.pdf
Data/Pdfs/Full text PDF/#17753 - BigLotteryFund 2018.pdf
Data/Pdfs/Full text PDF/#9983 - Werner 2009.pdf
Data/Pdfs/Full text PDF/#9450 - Osei-Kyei 2019.pdf
Data/Pdfs/Full text PDF/#17831 - Airaksinen 2020.pdf
Data/Pdfs/Full text PDF/#5495 - Nalli 2007.pdf
Data/Pdfs/Full text PDF/#17749 - ReesCe

In [10]:
pdf_files_df

,filename,number,author,year
0,#8561 - Skopec 2018.pdf,#8561,Skopec,2018
1,#6580 - Sherry 2012.pdf,#6580,Sherry,2012
2,#5545 - Basinga 2010.pdf,#5545,Basinga,2010
3,#15791 - Acevedo 2016.pdf,#15791,Acevedo,2016
4,#2065 - Storeng 2014.pdf,#2065,Storeng,2014
...,...,...,...,...
1388,#2582 - Mshana 2018.pdf,#2582,Mshana,2018
1389,#8332 - Robinson 2009.pdf,#8332,Robinson,2009
1390,#4475 - Blau 2010.pdf,#4475,Blau,2010
1391,#8933 - Duchoslav 2019.pdf,#8933,Duchoslav,2019


### List PDFs with no id number and extract filename information

In [11]:
file_names = []
file_authors = []
file_years = []
file_titles = []
for f in os.listdir(pdf_files_folder):
    if os.path.isfile(os.path.join(pdf_files_folder, f)):
        if re.match('[^# ].*\.pdf', f, re.IGNORECASE):
#             print(f)
            file_names.append(f)
    
            if re.search('-[0-9]{4}-', f):
                temp_auth = re.findall('.*-[0-9]{4}-', f)[0][:-6]
            else:
                temp_auth = ""
            file_authors.append(temp_auth)
            
            if re.search('-[0-9]{4}-',f):
                temp_year = re.findall('-[0-9]{4}-',f)[0].strip("-")
            else:
                temp_year = ""
            file_years.append(temp_year) 
            
            if re.search('-[0-9]{4}-',f):
                temp_title = re.findall('-[0-9]{4}-.+',f)[0][6:-4]
            else:
                temp_title = ""
            file_titles.append(temp_title)
            
pdf_files_df2 = pd.DataFrame()
pdf_files_df2['filename'] = file_names
pdf_files_df2['author'] = file_authors
pdf_files_df2['year'] = file_years
pdf_files_df2['Study'] = [simplify(el[0].replace(" ","")+' '+el[1]) for el in list(zip(file_authors, file_years))]
pdf_files_df2['title'] = [simplify(title) for title in file_titles]

### Assign pdf files to articles

In [12]:
pdf_file_list = []

for idx,row in texts_df[['Title','Authors','Study','Published Year','Covidence #']].iterrows():
    if len(pdf_files_df[pdf_files_df['number'].str.lower() == row['Covidence #'].lower()]) > 0:
        pdf_file_list.append(pdf_files_df[pdf_files_df['number'].str.lower() == row['Covidence #'].lower()]['filename'].tolist()[0])
    elif (len(pdf_files_df2[pdf_files_df2['Study'].str.lower() == row['Study'].lower()]) > 0):
        temp = ""
        for _, sample in pdf_files_df2[pdf_files_df2['Study'].str.lower() == row['Study'].lower()].iterrows():
            if row['Title'][:4].lower() == sample['title'][:4].lower():
                temp = sample["filename"]
                pdf_file_list.append(sample["filename"])
                break
        if temp == "":
            pdf_file_list.append('')
    else:
        pdf_file_list.append('')
texts_df['pdf_file'] = pdf_file_list

In [13]:
print('Articles with PDF assigned')
print(len(texts_df[texts_df['pdf_file'] != '']))

Articles with PDF assigned
1727


### Assign pdf files that have not been assigned to articles

In [14]:
set1 = set(texts_df['pdf_file'].tolist())
set2 = set(pdf_files_df2['filename'].tolist())

In [15]:
for row in set2.difference(set1):
    temp = pdf_files_df2[pdf_files_df2['filename'] == row]["title"].iloc[0].lower()
    temp = temp.replace("_",":")
    temp_name = pdf_files_df2[pdf_files_df2['filename'] == row]["Study"].iloc[0].split()
    if len(temp_name) == 0:
        continue
    temp_name = temp_name[0].lower() # get name from study
    
    # for pdfs not assigned but have matched title with a article
    if len(texts_df[texts_df.Title.str.match(temp, flags=re.IGNORECASE)])>0:
        # if the article has not assigned any pdfs and the Study or Authors submatch with the pdfs author. 
        if (texts_df[texts_df.Title.str.match(temp, flags=re.IGNORECASE)]["pdf_file"].iloc[0]=="") and ((
        temp_name in texts_df[texts_df.Title.str.match(temp, flags=re.IGNORECASE)]["Authors"].iloc[0].lower()) or 
            (temp_name in texts_df[texts_df.Title.str.match(temp, flags=re.IGNORECASE)]["Study"].iloc[0].lower())):
            print(row, temp_name, texts_df[texts_df.Title.str.match(temp, flags=re.IGNORECASE)]["Study"].iloc[0])
            texts_df.loc[texts_df.Title.str.match(temp, flags=re.IGNORECASE), 'pdf_file'] = row
#             print(texts_df[texts_df.Title.str.match(temp)]["pdf_file"].iloc[0], pdf_files_df2[pdf_files_df2['filename'] == row]["filename"].iloc[0])

Liang-2020-Beyond Pay‐for‐Performance_ Can Dif.pdf liang LLLiang 2020
da Costa e Silv-2017-Can the risk in public-pr.pdf dacostaesilv daCostaeSilva 2017
Jae‐jin-2020-What makes hybrid insourcing succ.pdf jaejin JaejinYang 2020
Janice Johnson-2007-For-Profit Welfare_ Contra.pdf janicejohnson JaniceJohnsonDias 2007
Turcotte-Trembl-2016-Does performance-based fi.pdf turcotte-trembl Turcotte-Tremblay 2016
Radies Kuspriha-2020-Local government and publ.pdf radieskuspriha RadiesKusprihantoPurbo 2020
Vanessa Crossgr-2019-Pay for Success_ Diffusio.pdf vanessacrossgr VanessaCrossgroveFry 2019
Virginia Rodríg-2017-Efectos de las alianzas p.pdf virginiarodrig VirginiaRodriguezNunodelaRosa 2017
Qureshi-2016-The role of cost-effectiveness an.pdf qureshi QureshiNadeem 2016
Sabbi-2020-The “Numbers Game”_ Strategic React.pdf sabbi SabbiMatthew 2020
William Casey-2020-Performance Funding and His.pdf williamcasey WilliamCaseyBoland 2020
Nicholas Simcik-2018-Seeing Like a City‐State_.pdf nicholassimcik 

In [16]:
print('Articles with PDF assigned')
print(len(texts_df[texts_df['pdf_file'] != '']))

Articles with PDF assigned
1740


### Save articles list with PDF filename information

In [17]:
texts_df.to_json(output_file_json,orient="records", force_ascii=False)

# Step2: extract text from pdfs

In [7]:
# pdf_files_folder = 'Data/Pdfs/all_pdfs'
pdf_files_folder = 'Data/Pdfs/Full text PDF'
pdf_files_txt_folder = 'Data/Pdfs/all_pdfs_txt'

input_file_json = 'step1_output.json'
output_file_json = 'step2_output.json'

In [8]:
texts_df = pd.read_json(input_file_json, orient="records")

In [9]:
print('Number of pdfs to extract content')
print(len(set(texts_df['pdf_file'].tolist())))

Number of pdfs to extract content
1734


### Dump PDF content to TXT file

In [11]:
exceptions = []
for f in tqdm.tqdm(texts_df['pdf_file']):
    if f != '' and os.path.isfile(os.path.join(pdf_files_folder,f)):
        try:
            if os.path.isfile(os.path.join(pdf_files_txt_folder, f[:-3]+'txt')):
                continue
            txt_content = extract_text(os.path.join(pdf_files_folder,f))
            with open(os.path.join(pdf_files_txt_folder, f[:-3]+'txt'), 'w') as fw:
                fw.write(txt_content)
        except:
            print(f)
            exceptions.append(f)

 23%|██▎       | 453/1952 [23:01<4:26:55, 10.68s/it] /opt/anaconda3/envs/py38/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='Data/Pdfs/Full text PDF/#12250 - Chiroleu-Assouline 2012.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
 36%|███▌      | 704/1952 [47:28<51:51,  2.49s/it]   /opt/anaconda3/envs/py38/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='Data/Pdfs/Full text PDF/#12653 - Montague 2003.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
 39%|███▊      | 754/1952 [50:28<4:19:32, 13.00s/it]/opt/anaconda3/envs/py38/lib/python3.8/site-packages/pdfminer/pdfpage.

### Extract References and Acknowledgements from TXT files

In [12]:
author = r"(?:[A-Z][0-9A-Za-z'`\?-]+)"
etal = r"(?:et al\.?)"
vol = r"(?:Vol\.? [0-9]+,? No\.? [0-9]+)"
additional = f"(?:,? (?:(?:and |& )?{author}|{etal}|{vol}))"
date_num = "(?:January|February|March|April|May|June|July|August｜September｜October｜November｜December)*[, ]*[0-9]{0,2}"
year_num = "(?:19|20)[0-9][0-9] *[–-]? *(?:(?:19|20)[0-9][0-9])?"
page_num = "(?:p+\.? [0-9]+)?"  # Always optional
year = fr"(?:, *{date_num}[, ]*{year_num}[, ]*{page_num}| *\({date_num}{year_num}{page_num}\))"
regex_references_incontent = fr'\b(?!(?:Although|Also)\b){author}{additional}*{year}'

regex_references_end = r"(?i:(?:Reference|references|References|Referências|Bibliography|Endnotes|Literatur|Bibliografia|Bibliografía|KIRJALLISUUS|Literature):.+)"
regex_references_end2 = r"(?!(?<=(?i:Terms of) )|(?<=(?i:cheme))|(?<=(?i:Annotated) )|(?<=(?i:the) )|(?<=(?i:Funding) )|(?<=(?i:end) )|(?<=(?i:Available) ))(?:Reference|references|References|Referências|Referencias|REFERENCE|REFERENCES|Bibliography|bibliography|Bibliografia|Bibliografía|BIBLIOGRAPHY|Endnotes|ENDNOTES|Literatur|KIRJALLISUUS|Sources|Literature|NOTES|Notes) (?!(?i:Review|Cited｜Case|group|PRICING|guide|Board|list|Committee|Curriculum|Note|Appendix|Annex|Annexes|Appendices|Applicability|Acknowledgement|Manual|Boxes|Code|Codes|Effectiveness|new|index|Laboratory|Abstract|Under|Patient|[.]+)) ?[^a-zA-Z]*(?:[A-Z](?:(?!(?:\. |\.){10,}).)+)$"
case1 = "R\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case2 = "[rR]\s*E\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case3 = "[rR]\s*[eE]\s*F\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case4 = "[rR]\s*[eE]\s*[fF]\s*E\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case5 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*R\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case6 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*E\s*[nN]\s*[cC]\s*[eE]\s*[sS]?.+"
case7 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*N\s*[cC]\s*[eE]\s*[sS]?.+"
case8 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*C\s*[eE]\s*[sS]?.+"
case9 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*E\s*[sS]?.+"
case10 = "[rR]\s*[eE]\s*[fF]\s*[eE]\s*[rR]\s*[eE]\s*[nN]\s*[cC]\s*[eE]\s*S.+"
case11 = "S\s*E\s*C\s*N\s*E\s*R\s*E\s*F\s*E\s*R.+"
regex_references_end3 = fr"{case1}|{case2}|{case3}|{case4}|{case5}|{case6}|{case7}|{case8}|{case9}|{case10}|{case11}"
regex_acknowledgment = r"((?:Acknowledgments|Acknowledgements|ACKNOWLEDGMENTS|ACKNOWLEDGEMENTS|Acknowledgement |Acknowledge )(?!(?i: Executive Summary)):? ?[—.(]?(?:\(cid:2\))? ?[a-zA-Z].+?)(?:(?i:Executive Summary|Table of Contents|Contents|Introduction|PREAMBLE|Highlights|Overview|Contributions))"

full_texts_list = []
references_incontent = []
references_end = []
acknowledgments = []

for f in tqdm.tqdm(texts_df['pdf_file']):
    if f != '' and os.path.isfile(os.path.join(pdf_files_txt_folder,f[:-3]+'txt')):
#         print(f)
        with open(os.path.join(pdf_files_txt_folder,f[:-3]+'txt'), 'r') as fr:
            txt_content = fr.read()
            txt_content = re.sub("\n+", " ", txt_content)
            txt_content = re.sub("\s+", " ", txt_content)
            
            temp = re.findall(regex_references_end, txt_content)
            if len(temp) > 0:
                temp = temp[0]
            else:
                temp = re.findall(regex_references_end2, txt_content)
                if len(temp) > 0:
                    temp = temp[0]
                else:
                    temp = re.findall(regex_references_end3, txt_content)
                    if len(temp) > 0:
                        temp = temp[0]
                    else:
                        temp = ""
            if re.search("[.]{20,}", temp):
                temp = ""
            
            matches = re.findall(regex_references_incontent, txt_content)
            if len(matches) > 0:
                matches = matches
            else:
                matches = []
            
            ack_matches = ''
            ack_index = [(m.start(0), m.end(0)) for m in re.finditer("(?:Acknowledgments|Acknowledgements|ACKNOWLEDGMENTS|ACKNOWLEDGEMENTS|Acknowledgement |Acknowledge )", txt_content)]
            if len(ack_index) > 0:
                ack_index = ack_index[-1][0]
                if ack_index / len(txt_content) <= 0.3: # acknowledgment at the beginning
                    ack_matches = re.findall(regex_acknowledgment, txt_content)
                    if len(ack_matches) > 0:
                        ack_matches = ack_matches[0]
                    else:
                        ack_matches = ""
                else: # acknowledgment at the end
                    ack_matches = re.findall("(?:Acknowledgments|Acknowledgements|ACKNOWLEDGMENTS|ACKNOWLEDGEMENTS|Acknowledgement |Acknowledge ):? ?[—.(]?(?:\(cid:2\))? ?[a-zA-Z].+", txt_content)
                    if len(ack_matches) > 0:
                        ack_matches = ack_matches[0]
                    else:
                        ack_matches = ""

        full_texts_list.append(txt_content)
        references_end.append(temp)
        references_incontent.append(matches)
        acknowledgments.append(ack_matches)
    else:
        full_texts_list.append("")
        references_end.append("")
        references_incontent.append([])
        acknowledgments.append("")
        
texts_df['pdf_txt'] = full_texts_list
texts_df['references_incontent'] = references_incontent
texts_df['references_end'] = references_end
texts_df['acknowledgments'] = acknowledgments

100%|██████████| 1952/1952 [02:51<00:00, 11.41it/s]


### Save Article list including the text of the assigned PDF

In [13]:
texts_df.to_json(output_file_json, orient="records", force_ascii=False)

# Step3: search for mentioned Indigo Projects and Organisations in each article

In [14]:
data_dir = 'Data/Indigo'

input_file_json = 'step2_output.json'
output_file_json = 'step3_output.json'

### List Projects and Organisations from Indigo

In [15]:
projects_cols = ['ID','Impact Bond Name - (Value)','Alternative Names - (Value)','Organisations','Countries']
projects_df = pd.read_csv(os.path.join(data_dir,'projects.csv'),usecols=projects_cols)
organisations_cols = ['ID','Name', 'Trading Name', 'Translated Official Name - English',
                      'Translated Official Name - Source Language','ORG-Ids - Primary','Country']
organisations_df = pd.read_csv(os.path.join(data_dir,'organisations.csv'),usecols=organisations_cols)
projects_delivery_locations_cols = ['Project ID','Location - Name - (Value)','Location - Country - (Value)']
projects_delivery_locations_df = pd.read_csv(os.path.join(data_dir,'all_data_per_data_type_csv',
                                                          'projects_delivery_locations.csv'),
                                             usecols=projects_delivery_locations_cols)

In [16]:
projects_list = ['Kobe SIB'] + list(set(projects_df['Impact Bond Name - (Value)'].tolist())-{np.nan}) + \
    list(set(projects_df['Alternative Names - (Value)'].tolist())-{np.nan, "Addaction", "Bexley", "Kirklees"})
organisations_list = list(set(organisations_df['Name'].tolist())-{np.nan}) + \
    list(set(organisations_df['Trading Name'].tolist())-{np.nan}) + \
    list(set(organisations_df['Translated Official Name - English'].tolist())-{np.nan}) + \
    list(set(organisations_df['Translated Official Name - Source Language'].tolist())-{np.nan})
organisations_list = [str(org) for org in organisations_list]
organisations_list = [org for org in organisations_list if org != 'nan']
organisations_list = [org for org in organisations_list if org != '[Anonymous]']

In [79]:
projects_list = [project.strip() for project in projects_list]
projects_list

NameError: name 'projects_list' is not defined

In [25]:
len(organisations_list)

1101

### Read articles list with content

In [26]:
texts_df = pd.read_json(input_file_json,orient="records")
texts_df.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,Ref,Covidence #,Study,Notes,Tags,pdf_file,pdf_txt,references_incontent,references_end,acknowledgments
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,None,#17157,JonKlinepeter 2019,Felix-Anselm van Lier (2021-08-25 21:15:36)(Se...,Grey literature - general web search; Qualitat...,#17157 - JonKlinepeter 2019.pdf,JULY 2019 Innovation in Child Welfare Through ...,[],"Endnotes 1 Kelly J, Heimpel H, Loudenback J, R...",
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,None,#17224,JamesRonicle 2016,Felix-Anselm van Lier (2021-08-21 07:20:35)(Se...,Grey literature - general web search,#17224 - JamesRonicle 2016.pdf,Commissioning Better Outcomes Fund Evaluation...,"[Ronicle et al, 2014, DCLG, 2015, DWP, 2016, O...",,
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,None,#17223,JamesRonicle 2019,Felix-Anselm van Lier (2021-08-21 07:15:53)(Se...,Grey literature - general web search; Independ...,#17223 - JamesRonicle 2019.pdf,Commissioning Better Outcomes Fund Evaluation...,"[Social Finance, 2016, , Moffat et al, 2017, L...",,
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,None,#17284,Warner 2018,Lilly Monk (2021-08-21 02:07:03)(Select): 3x c...,Education; Grey literature - general web searc...,#17284 - Warner 2018.pdf,Journal of Urban Affairs ISSN: (Print) (Online...,"[Warner (2020), Pequeneza, 2018, Fraser, Tan, ...",Notes 1. Contract and loan agreements are list...,Acknowledgments Funding This research was supp...
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,None,#4965,McBeath 2006,Lilly Monk (2021-08-21 01:59:55)(Select): Outc...,Qualitative analysis,McBeath-2006-Nonprofit Adaptation to Performan...,Administration in Social Work ISSN: 0364-3107 ...,[Bowen McBeath MSW & William Meezan DSW (2006)...,NOTES 1. Few empirical studies have estimated ...,


In [27]:
# remove references and acknowledgments
for idx, row in tqdm.tqdm(texts_df.iterrows()):
    if row['references_end'] in row['acknowledgments']:
        temp = row['acknowledgments']
    elif row['acknowledgments'] in row['references_end']:
        temp = row['references_end']
        
    if temp in row['pdf_txt']:
        texts_df.at[idx, 'pdf_txt'] = texts_df.at[idx, 'pdf_txt'].replace(temp, "")
    if row['acknowledgments'] in row['pdf_txt']:
        texts_df.at[idx, 'pdf_txt'] = texts_df.at[idx, 'pdf_txt'].replace(row['acknowledgments'], "")
    for ref in row['references_incontent']:
        texts_df.at[idx, 'pdf_txt'] = texts_df.at[idx, 'pdf_txt'].replace(ref, "")

1952it [00:40, 48.22it/s] 


In [28]:
texts_df['id'] = texts_df.index
texts_df['txt'] = texts_df[['Title','Abstract','pdf_txt']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract'])+'\n'+str(row['pdf_txt']), axis=1)
texts_df

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,Study,Notes,Tags,pdf_file,pdf_txt,references_incontent,references_end,acknowledgments,id,txt
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,JonKlinepeter 2019,Felix-Anselm van Lier (2021-08-25 21:15:36)(Se...,Grey literature - general web search; Qualitat...,#17157 - JonKlinepeter 2019.pdf,JULY 2019 Innovation in Child Welfare Through ...,[],"Endnotes 1 Kelly J, Heimpel H, Loudenback J, R...",,0,Innovation In Child Welfare Through Pay For Su...
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,JamesRonicle 2016,Felix-Anselm van Lier (2021-08-21 07:20:35)(Se...,Grey literature - general web search,#17224 - JamesRonicle 2016.pdf,Commissioning Better Outcomes Fund Evaluation...,"[Ronicle et al, 2014, DCLG, 2015, DWP, 2016, O...",,,1,Commissioning Better Outcomes Fund Evaluation ...
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,JamesRonicle 2019,Felix-Anselm van Lier (2021-08-21 07:15:53)(Se...,Grey literature - general web search; Independ...,#17223 - JamesRonicle 2019.pdf,Commissioning Better Outcomes Fund Evaluation...,"[Social Finance, 2016, , Moffat et al, 2017, L...",,,2,Commissioning Better Outcomes Fund Evaluation ...
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,Warner 2018,Lilly Monk (2021-08-21 02:07:03)(Select): 3x c...,Education; Grey literature - general web searc...,#17284 - Warner 2018.pdf,Journal of Urban Affairs ISSN: (Print) (Online...,"[Warner (2020), Pequeneza, 2018, Fraser, Tan, ...",Notes 1. Contract and loan agreements are list...,Acknowledgments Funding This research was supp...,3,The RazorS Edge: Social Impact Bonds And The F...
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,McBeath 2006,Lilly Monk (2021-08-21 01:59:55)(Select): Outc...,Qualitative analysis,McBeath-2006-Nonprofit Adaptation to Performan...,Administration in Social Work ISSN: 0364-3107 ...,[Bowen McBeath MSW & William Meezan DSW (2006)...,NOTES 1. Few empirical studies have estimated ...,,4,"Nonprofit Adaptation to Performance-Based, Man..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,Innovation in Australian publicly operated pub...,"Saeed, Ali Mohammed; Duffield, Colin; Hui, Felix",It is challenging to identify or evaluate inno...,2019.0,None,INFRASTRUCTURE ASSET MANAGEMENT,6,3,166-177,None,...,Saeed 2019,Michael Gibson (2021-01-30 03:09:34)(Screen): ...,None,Saeed-2019-Innovation in Australian publicly o...,"Cite this article Saeed AM, Dufﬁeld C and Hui ...","[Regan et al., 2009, Gaiser, 2009, Dixon et al...",REFERENCES Akhmetshina E and Mustaﬁn A (2015) ...,Acknowledgements authors would like The acknow...,1947,Innovation in Australian publicly operated pub...
1948,Social impact bonds: The evolution of research...,"Broccardo, Eleonora; Mazzuca, Maria; Frigotto,...",This study reviews all the academic contributi...,2020.0,None,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,27,3,1316-1332,None,...,Broccardo 2020,None,None,Broccardo-2020-Social impact bonds_ The evolut...,Received: 31 March 2019 Revised: 13 November 2...,"[Lagarde, & Mays, 2018, Dixon-Woods et al., 20...",ENDNOTES 1 For an overview of impact investing...,,1948,Social impact bonds: The evolution of 

### Search for Projects and Organisations from Indigo

In [29]:
def txt_org_search(txt, orgs_list):
    temp_txt = txt
    found_orgs = []
    for org in orgs_list:
        if org == "ParentChild+":
            mentioned = re.findall(r"ParentChild\+", temp_txt)
        elif org == "Chances":
            mentioned = re.findall(r"\b(?!(?<=(?i:Life) ))Chances(?!(?i: Fund))\b", temp_txt)
        elif org == "Pause":
            mentioned = re.findall(r"(?!(?<=[\(]))Pause(?!(?i:[\)]|delivery))", temp_txt)
        elif org == "Opening Doors":
            temp_org = re.sub("[^a-zA-Z0-9 ]", "", simplify(str(org).strip()))
            temp_org = "[^a-zA-Z0-9 ]*".join([c for c in temp_org])
            mentioned = re.findall(fr"(?!(?<=Appendix to )){temp_org}", temp_txt)
        else:
            temp_org = re.sub("[^a-zA-Z0-9 ]", "", simplify(str(org).strip()))
            temp_org = "[^a-zA-Z0-9 ]*".join([c for c in temp_org])
            mentioned = re.findall(fr"\b{temp_org}\b", temp_txt)

        if mentioned:
            found_orgs.append((str(org), len(mentioned)))
#             found_orgs.append(str(org))
    return found_orgs

def txt_org_search_lower(txt, orgs_list):
    temp_txt = txt.lower()
    found_orgs = []
    for org in orgs_list:
        if org == "ParentChild+":
            mentioned = re.findall(r"parentchild\+", temp_txt)
        elif org == "Chances":
            mentioned = re.findall(r"\b(?!(?<=(?i:life) ))chances(?!(?i: fund))\b", temp_txt)
        elif org == "Pause":
            mentioned = re.findall(r"(?!(?<=[\(]))pause(?!(?i:[\)]|delivery))", temp_txt)
        elif org == "Opening Doors":
            temp_org = re.sub("[^a-zA-Z0-9 ]", "", simplify(str(org).strip())).lower()
            temp_org = "[^a-zA-Z0-9 ]*".join([c for c in temp_org])
            mentioned = re.findall(fr"(?!(?<=appendix to )){temp_org}", temp_txt)
        else:
            temp_org = re.sub("[^a-zA-Z0-9+ ]", "", simplify(str(org).strip())).lower()
            temp_org = "[^a-zA-Z0-9 ]*".join([c for c in temp_org])
            mentioned = re.findall(fr"\b{temp_org}\b", temp_txt)
        if mentioned:
            found_orgs.append((str(org), len(mentioned)))
#             found_orgs.append(str(org))
    return found_orgs

### Search for Projects from Indigo

In [30]:
txt_projects = []
elements = []
for idx, row in tqdm.tqdm(texts_df[['txt']].iterrows()):
    temp = txt_org_search(row["txt"], projects_list)
    txt_projects.append(temp)
    elements += temp
texts_df['txt_projects'] = txt_projects
print(len(set(elements)))
set(elements)

1952it [15:50,  2.05it/s]

168


{('3SC Capitalise', 1),
 ('3SC Capitalise', 6),
 ('Alameda County Justice Restoration Project', 1),
 ('Ambition', 1),
 ('Ambition', 2),
 ('Ambition', 5),
 ('Ambition', 12),
 ('Aspire', 1),
 ('Aspire', 2),
 ('Aspire', 3),
 ('Aspire', 5),
 ('Aspire', 10),
 ('Aspire', 17),
 ('Aspire', 31),
 ('Aspire', 40),
 ('Big Picture Learning, Doncaster', 1),
 ('Buzinezzclub (Eindhoven)', 1),
 ('Buzinezzclub (Rotterdam)', 1),
 ('Buzinezzclub (Rotterdam)', 3),
 ('Cameroon Cataract Bond', 1),
 ('Cameroon Cataract Bond', 2),
 ('Cameroon Cataract Bond', 3),
 ('Cameroon Cataract Bond', 13),
 ('Cameroon Cataract Bond', 19),
 ('Cameroon Cataract Bond', 25),
 ('Chances', 1),
 ('Chances', 74),
 ('Connecticut Family Stability Pay for Success Project', 1),
 ('Denver Housing to Health Initiative', 1),
 ('Depaul', 1),
 ('Depaul', 2),
 ('Depaul', 3),
 ('Depaul', 6),
 ('Depaul', 12),
 ('Duo for a job', 1),
 ('Educate Girls ', 1),
 ('Educate Girls ', 2),
 ('Educate Girls ', 3),
 ('Educate Girls ', 4),
 ('Educate Girl

In [31]:
projects_list_lower = ["cornwall frequent attenders", 
                       "duo for a job",
                       "Kobe City SIB for Preventing Severe Diabetic Nephropathy",
                       "Mental Health and Employment Partnership (MHEP) Staffordshire, Haringey, and Tower Hamlets",
                       "The Cuyahoga Partnering for Family Success Program"]

In [32]:
txt_projects_lower = []
elements = []
for idx, row in tqdm.tqdm(texts_df[['txt']].iterrows()):
    temp = txt_org_search_lower(row["txt"],  projects_list_lower)
    txt_projects_lower.append(temp)
    elements += temp
texts_df['txt_projects_lower'] = txt_projects_lower
print(len(set(elements)))
set(elements)

1952it [00:19, 97.62it/s] 


6


{('The Cuyahoga Partnering for Family Success Program', 1),
 ('cornwall frequent attenders', 1),
 ('duo for a job', 1),
 ('duo for a job', 2),
 ('duo for a job', 7),
 ('duo for a job', 10)}

### Search for Orgs from Indigo

In [33]:
txt_orgs = []
elements = []
for idx, row in tqdm.tqdm(texts_df[['txt']].iterrows()):
    temp = txt_org_search(row["txt"], set(organisations_list))
    txt_orgs.append(temp)
    elements += temp
texts_df['txt_orgs'] = txt_orgs
print(len(set(elements)))
set(elements)

1952it [1:49:38,  3.37s/it]

1306


{('Linklaters', 4),
 ('Enterprise Community Partners', 2),
 ('Big Society Capital', 1),
 ('ES', 16),
 ('Cabinet Office', 10),
 ('Article 1', 1),
 ("St Basil's", 1),
 ('Social Impact Finance', 2),
 ('FR', 2),
 ('Park City School District', 1),
 ('CCP', 3),
 ('MaRS Centre for Impact Investing', 14),
 ('DC Water and Sewer Authority', 3),
 ('Commonwealth of Massachusetts', 4),
 ('Chesapeake Research Associates', 1),
 ("Children's Express", 1),
 ('Compartamos con Colombia', 3),
 ('Big Society Capital', 14),
 ('US Department of Labor', 4),
 ('Bridges Fund Management', 3),
 ('Depaul', 1),
 ('Instiglio', 32),
 ('Kuepa', 3),
 ("Children's Literacy Charity", 1),
 ('The Cleveland Foundation', 1),
 ("Lit'l Scholars", 1),
 ('Stibbe', 3),
 ('Worcestershire County Council', 8),
 ('Shelter', 1),
 ('Conrad N. Hilton Foundation', 1),
 ('Prevista', 3),
 ('ES', 6),
 ('Ventura County', 5),
 ('Netri Foundation', 2),
 ('Alameda County', 25),
 ('Mayday Trust', 4),
 ('Nurse-Family Partnership', 3),
 ('Social F

### Save Article list including the Indigo projects and orgs, mentioned at least two times in an article

In [34]:
threshold = 2
final_projects = []
for idx, row in tqdm.tqdm(texts_df[['txt_projects', 'txt_projects_lower']].iterrows()):
    temp = []
    for project in row["txt_projects"]:
        if project[1] >= threshold:
            temp.append(project[0])
    for project in row["txt_projects_lower"]:
        if project[1] >= threshold:
            temp.append(project[0])
    final_projects.append(sorted(set(temp)))
texts_df['mentioned_projects'] = final_projects

final_orgs = []
for idx, row in tqdm.tqdm(texts_df[['txt_orgs']].iterrows()):
    temp = []
    for org in row["txt_orgs"]:
        if org[1] >= threshold:
            temp.append(org[0])
    final_orgs.append(sorted(set(temp)))
texts_df["mentioned_orgs"] = final_orgs

1952it [00:00, 5364.86it/s]
1952it [00:00, 6234.83it/s]


In [35]:
texts_df.to_json(output_file_json,orient="records", force_ascii=False)

# Step4: extract named entities in each article

In [12]:
input_file_json = 'step3_output.json'
texts_df = pd.read_json(input_file_json,orient="records")

In [6]:
sorted([len(i) for i in texts_df['pdf_txt'].to_list()])[::-1]

[8220034,
 1940469,
 1439604,
 1400548,
 1203403,
 967671,
 895795,
 877468,
 862462,
 803217,
 794091,
 765954,
 741441,
 741370,
 741370,
 734082,
 694106,
 692297,
 689847,
 677409,
 671774,
 659271,
 650104,
 644182,
 643780,
 639763,
 636083,
 634848,
 632331,
 629938,
 618428,
 609278,
 603786,
 593448,
 585114,
 563239,
 562274,
 549372,
 540218,
 535662,
 526393,
 523069,
 517440,
 514700,
 508298,
 506317,
 505335,
 498013,
 498013,
 495406,
 491659,
 485938,
 484266,
 484242,
 475956,
 462255,
 456623,
 454132,
 453902,
 450571,
 449223,
 440360,
 434731,
 429361,
 428061,
 421173,
 418422,
 417693,
 417139,
 417139,
 413414,
 413414,
 412932,
 398907,
 395805,
 395769,
 390324,
 387140,
 383405,
 383096,
 373828,
 373334,
 370954,
 366108,
 363164,
 361718,
 361234,
 361234,
 357887,
 353913,
 350814,
 345144,
 345144,
 342622,
 334044,
 333578,
 330097,
 328889,
 328373,
 328290,
 328150,
 323791,
 323791,
 320709,
 320504,
 317839,
 312208,
 311204,
 310888,
 310518,
 3093

### load NER models

In [ ]:
# !pip install spacy==3.0
# !pip install -U flair
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_trf

In [7]:
import spacy
spacy.require_gpu() 
nlp_lg = spacy.load("en_core_web_lg", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
# If the previous line does not work:
# import en_core_web_lg
# nlp_lg = en_core_web_lg.load()
nlp_lg.max_length = 1500000

In [8]:
spacy.require_gpu() 
nlp_trf = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
# If the previous line does not work:
# import en_core_web_trf
# nlp_trf = en_core_web_trf.load()
nlp_trf.max_length = 1500000

In [10]:
# for flair ner model
import torch
import flair
from flair.data import Sentence
from flair.models import SequenceTagger
flair.device = torch.device('cuda:0')
model = SequenceTagger.load("flair/ner-english-ontonotes-large")

2022-04-20 16:46:31,885 loading file flair/ner-english-ontonotes-large


FileNotFoundError: [Errno 2] No such file or directory: 'flair/ner-english-ontonotes-large'

### Extract named entities

In [9]:
# ner_labels = ['PERSON','ORG','GPE','LOC','DATE','NORP','LAW','MONEY']
ner_labels = ['ORG','GPE','LAW']
# exclude_list = ['nan']
exclude_list = ['']

In [10]:
# for en_core_web_lg and en_core_web_trf
def txt_selected_ners(text, ner_labels, exclude_list, model):
    result = []
    contents = text
    for patex in exclude_list:
        contents = re.sub(patex, '', contents)    

    step = 512
    tokens = contents.split()
    for i in range(0, len(tokens), step):
        sentence = " ".join(tokens[i:i+step])
        doc = model(sentence)
        # with some ner analysers, it works better using .lower()
        if len(doc.ents) > 0:
            for ent in doc.ents:
                if ent.label_ in ner_labels:
                    start = ent.start - 10
                    end = ent.end + 10
                    if start < 0:
                        start = 0
                    context = doc[start:end]
                    result.append((ent.text, ent.label_, context))
    return result

# for flair
def txt_selected_ners_flair(text, ner_labels, exclude_list):
    result = []
    contents = text
    for patex in exclude_list:
        contents = re.sub(patex, '', contents)    
        
    step = 512
    tokens = contents.split()
    # sentences = [Sentence(" ".join(tokens[i:i+step])) for i in range(0, len(tokens), step)]
    for i in range(0, len(tokens), step):
        try:
            sentences = [Sentence(" ".join(tokens[i:i+step]))]
            model.predict(sentences, mini_batch_size=16)

            # with some ner analysers, it works better using .lower()
            for sentence in sentences:
                for entity in sentence.get_spans('ner'):
                    start = entity.tokens[0].idx - 1 - 10
                    end = entity.tokens[-1].idx + 10
                    if start < 0:
                        start = 0
                    context = to_plain_string(sentence.tokens[start:end])
                    result.append((entity.text, entity.labels[0].value, context))
                    del entity
                del sentence
            del sentences
            torch.cuda.empty_cache()
        except:
            continue
    return result

def to_plain_string(tokens):
    plain = ""
    for token in tokens:
        plain += token.text
        if token.whitespace_after > 0:
            plain += token.whitespace_after * " "
    return plain.rstrip()

In [11]:
for idx,row in texts_df.iterrows():
    if idx < 1:
        print(txt_selected_ners(row['txt'], ner_labels, exclude_list, nlp_trf))

[('Safe Families For Children', 'ORG'), ('United States', 'GPE'), ('every year,1', 'DATE'), ('Pay for Success', 'ORG'), ('Safe Families for Children', 'ORG'), ('Safe Families', 'ORG'), ('Safe Families', 'ORG'), ('LISC', 'ORG'), ('Safe Families', 'ORG'), ('fall 2018', 'DATE'), ('Safe Families', 'ORG'), ('LISC', 'ORG'), ('Pay for SuccessIn', 'ORG'), ('2017', 'DATE'), ('Local Initiatives Support Corporation', 'ORG'), ('LISC', 'ORG'), ('Safe Families for Children', 'ORG'), ('Safe Families', 'ORG'), ('LISC', 'ORG'), ('2016', 'DATE'), ('the Social Innovation Fund', 'ORG'), ('SIF', 'ORG'), ('JULY', 'DATE'), ('Jon Klinepeter', 'PERSON'), ('Anna Smukowski', 'PERSON'), ('Samantha Creath', 'PERSON'), ('Terry Gillen', 'PERSON'), ('United States', 'GPE'), ('every year,1', 'DATE'), ('Safe Families for Children', 'ORG'), ('Safe Families', 'ORG'), ('Safe Families', 'ORG'), ('LISC', 'ORG'), ('Safe Families', 'ORG'), ('fall 2018', 'DATE'), ('Safe Families', 'ORG'), ('LISC', 'ORG'), ('2017', 'DATE'), ('L

In [12]:
texts_df['ALL_NER_lg'] = texts_df['txt'].progress_apply(lambda x: txt_selected_ners(x,ner_labels,exclude_list, nlp_lg))
texts_df.to_json("ner_output_lg.json", orient="records", force_ascii=False)
texts_df['ALL_NER_trf'] = texts_df['txt'].progress_apply(lambda x: txt_selected_ners(x, ner_labels, exclude_list, nlp_trf))
texts_df.to_json("ner_output_lg_trf.json", orient="records", force_ascii=False)
texts_df['ALL_NER_flair'] = texts_df['txt'].progress_apply(lambda x: txt_selected_ners_flair(x,ner_labels, exclude_list))
texts_df.to_json("ner_output_lg_trf_flair.json", orient="records", force_ascii=False)

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4612 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
texts_df.to_json("ner_output.json", orient="records", force_ascii=False)

### Post-Processing entities

In [18]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [19]:
texts_df = pd.read_json("ner_output.json", orient="records")
texts_df.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,id,txt,txt_projects,txt_projects_lower,txt_orgs,mentioned_projects,mentioned_orgs,ALL_NER_lg,ALL_NER_trf,ALL_NER_flair
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,0,Innovation In Child Welfare Through Pay For Su...,[[Connecticut Family Stability Pay for Success...,[],"[[The Kresge Foundation, 1], [Youth Services, ...",[],"[Kresge Foundation, Nonprofit Finance Fund, So...","[[Safe Families, ORG, Despite positive interim...","[[United States, GPE, Safe Families For Childr...","[[United States, GPE, Safe Families For Childr..."
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,1,Commissioning Better Outcomes Fund Evaluation ...,[],[],"[[local providers, 2], [Newcastle Gateshead CC...",[],"[ATQ, Action for Children, Age UK, Cabinet Off...","[[England, GPE, bonds (SIBs) in England, andth...","[[The CBO Fund, ORG, Outcomes Fund Evaluation ...","[[The CBO Fund, ORG, Outcomes Fund Evaluation ..."
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,2,Commissioning Better Outcomes Fund Evaluation ...,"[[West London Zone, 9], [Mayday Inspire, 1], [...",[],"[[Birmingham City Council, 4], [The National L...",[West London Zone],"[ATQ, Birmingham City Council, Bridges Fund Ma...","[[the Commissioning Better Outcomes, ORG, is a...","[[Ecorys UK, ORG, ) Fund Evaluation undertaken...","[[Commissioning Better Outcomes Fund, ORG, Com..."
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,3,The RazorS Edge: Social Impact Bonds And The F...,[[South Carolina Nurse-Family Partnership Pay ...,[],"[[United Way of Salt Lake, 2], [Granite School...",[South Carolina Nurse-Family Partnership Pay f...,"[Granite School District, J.B. and M.K. Pritzk...","[[U.S., GPE, In a growing number of U.S. citie...","[[U.S., GPE, In a growing number of U.S. citie...","[[U.S., GPE, In a growing number of U.S. citie..."
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,4,"Nonprofit Adaptation to Performance-Based, Man...",[],[],[[University of Michigan School of Social Work...,[],[],"[[Nonprofit Adaptation, ORG, Nonprofit Adaptat...","[[Michigan, GPE, , Managed Care Contracting in...","[[HAWORTH DOCUMENT DELIVERY CENTER, ORG, COPIE..."


In [20]:
def postprocess_ners(ent_list):
    result = [] 
    for ent in ent_list:
        if ent[1] == "DATE" or ent[1] == "MONEY" or ent[1] == "LAW":
            temp = re.sub("^[^0-9a-zA-Z]+", "", ent[0])
            temp = re.sub("[^0-9a-zA-Z]+$", "", temp)
        else:
            temp = re.sub("^[^a-zA-Z]+", "", ent[0])
            temp = re.sub("[^a-zA-Z]+$", "", temp)
        result.append((temp, ent[0], ent[1], ent[2]))
    return result

In [21]:
# for ORG entities
def postprocess_orgs(ent_list):
    exclude_list = set(["et al", "ACKNOWLEDGEMENT", "ADMINISTRATION", "www", "co.uk", "com", "Gov",
                    'al.', "wiley", "ann", "arch", "archives", "press", "am j", "bmj", "bmc", "publ", 
                    "bull", "rev", "prev", "j", "journal", 'university', 'institute', 'faculty', 'review', 'annals', "et al", "http", "com", "Title",
                    "Article", "CHAPTER", "Annex", "Table", "Section", "Appendix", "Round", "Reference"])
    temp_list = []
    for ent in ent_list:
        if ent[2] != 'ORG':
            continue
        free = False
        for term in exclude_list:
            if re.search(fr"\b{term}\b", ent[0], re.IGNORECASE):
                free = True
        if not free:
            temp = re.sub("^[^a-zA-Z]+", "", ent[0]).strip()
            temp = re.sub("[^a-zA-Z]+$", "", temp).strip()
            temp = re.sub("( - )[a-z]", "", temp).strip()
            temp = re.sub("['’]s$", "", temp).strip()
            temp = re.sub("\s*['’]s", "’s", temp).strip()
            temp = re.sub(r"[^a-zA-Z0-9:&\-,.’ ]", "", temp).strip()
            temp = re.sub(r"\s*\.\s*", ".", temp).strip()
            temp = re.sub(r"^(for)", "", temp).strip()
            temp = re.sub(r"^(of)", "", temp).strip()
            temp = re.sub(r"\s+", " ", temp).strip()
            temp = re.sub(r"^the|^The", "", temp).strip()
            temp = re.sub(r"^Foundations?$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Organisations?$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Governments?$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Health$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^World$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Work$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Finance$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^\w\.\w$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"Ofce", "Office", temp, re.IGNORECASE).strip()
            if temp in stops:
                continue
            if (len(temp) >= 3 and not re.search(" Survey$|Survey of|Projects?", temp, re.IGNORECASE) and 
            not re.search("(^a|^an|^and|^any|^non|^that|^non-) ", temp, re.IGNORECASE) and 
            not re.search("(for a$|for$|of$|Fund$)", temp, re.IGNORECASE) and 
            not re.search("(Reports?$|Programmes?$|Programs?$|SIBs?$|Chapters?$| NA$|Schemes?$|Plans?$|Reform$|Bonds?$|Systems?$|Services?$Funds?$|Outcomes?$|Acts?$)", temp, re.IGNORECASE)) :
                temp_list.append((temp, ent[1], ent[3]))
    return temp_list

# for LOC entities
def postprocess_gpes(ent_list):
    exclude_list = set(["et al", "ACKNOWLEDGEMENT", "ADMINISTRATION", "www", "co.uk", "com", "Gov",
                    'al.', "wiley", "ann", "arch", "archives", "press", "am j", "bmj", "bmc", "publ", 
                    "bull", "rev", "prev", "j", "journal", 'university', 'institute', 'faculty', 'review', 'annals', "et al", "ACKNOWLEDGEMENT", "ADMINISTRATION", "www", "http", "com", "Title",
                    "Article", "CHAPTER", "Annex", "Table", "Section", "Appendix", "Round", "Reference"])
    temp_list = []
    for ent in ent_list:
        if ent[2] != 'GPE':
            continue
        free = False
        for term in exclude_list:
            if term.lower() in ent[0].lower():
                free = True
        if not free:
            temp = re.sub("^[^a-zA-Z]+", "", ent[0]).strip()
            temp = re.sub("[^a-zA-Z]+$", "", temp).strip()
            temp = re.sub("['’]s$", "", temp).strip()
            temp = re.sub("\s*['’]s", "’s", temp).strip()
            temp = re.sub(r"^the|^The", "", temp).strip()
            temp = re.sub(r"^\w\.\w$", "", temp, re.IGNORECASE).strip()
            if temp in stops:
                continue
            if len(temp) >= 3:
                temp_list.append((temp, ent[1], ent[3]))
    return temp_list

# for LAW entities
def postprocess_laws(ent_list):
    exclude_list = set(["et al", "ACKNOWLEDGEMENT", "ADMINISTRATION", "www", "co.uk", "com", "Gov",
                    'al.', "wiley", "ann", "arch", "archives", "press", "am j", "bmj", "bmc", "publ", 
                    "bull", "rev", "prev", "j", "journal", 'university', 'institute', 'faculty', 'review', 'annals', "et al", "ACKNOWLEDGEMENT", "ADMINISTRATION", "www", "http", "com", "Title",
                    "Article", "CHAPTER", "Annex", "Table", "Section", "Appendix", "Round", "Reference"])
    temp_list = []
    for ent in ent_list:
        if ent[2] != 'LAW':
            continue
        free = False
        for term in exclude_list:
            if term.lower() in ent[0].lower():
                free = True
        if not free:
            temp = re.sub("^[^0-9a-zA-Z]+", "", ent[0])
            temp = re.sub("[^0-9a-zA-Z]+$", "", temp)
            temp = re.sub(r"^Agreements?$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Decree$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Bond$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Treaty$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Policy$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Code$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Constitution$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Proposition$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Cth$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Directive$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^D.Act$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Official Plan$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Charter$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Convention$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^SIB$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^Act$", "", temp, re.IGNORECASE).strip()
            temp = re.sub(r"^\w\.\w$", "", temp, re.IGNORECASE).strip()
            if not re.search("^[A-Z]", temp) or re.search("[^a-zA-Z. ]", temp) or temp in stops:
                continue
            temp = re.sub(r"^the|^The", "", temp).strip()
            if (len(temp) >= 3 and 
            not re.search("(^a|^an|^and|^any|^non|^that|^part|^phase) ", temp, re.IGNORECASE) and
            not re.search("of$", temp, re.IGNORECASE)):
                temp_list.append((temp, ent[1], ent[3]))
    return temp_list

def filter_by_frequency(ent_list, frequency):
    unique_list = [ent[0] for ent in ent_list]
    temp_list = []
    for ent in ent_list:
        if unique_list.count(ent[0]) >= frequency:
            temp_list.append(ent)
    return temp_list

def filter_by_modelCount(row, entType, count_all, threshold):
    lg = row[entType+'_lg']
    trf = row[entType+'_trf']
    flair = row[entType+'_flair']
    union = lg+trf+flair
    temp_list = []
    for ent in set(union):
        if count_all[ent[0]] >= threshold:
            temp_list.append(ent)
    return temp_list


def ent_from_oneEntType(row, entType, type_all):
    target = row['pre_' + entType]
    temp_list = []
    for ent in target:
        if type_all.count(ent[0]) == 1:
            temp_list.append(ent)
    return temp_list

In [22]:
texts_df['ALL_NER_lg'] = texts_df['ALL_NER_lg'].progress_apply(lambda x: postprocess_ners(x))
texts_df['ALL_NER_trf'] = texts_df['ALL_NER_trf'].progress_apply(lambda x: postprocess_ners(x))
texts_df['ALL_NER_flair'] = texts_df['ALL_NER_flair'].progress_apply(lambda x: postprocess_ners(x))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [23]:
texts_df['ORG_lg'] = texts_df['ALL_NER_lg'].progress_apply(lambda x: postprocess_orgs(x))
texts_df['ORG_trf'] = texts_df['ALL_NER_trf'].progress_apply(lambda x: postprocess_orgs(x))
texts_df['ORG_flair'] = texts_df['ALL_NER_flair'].progress_apply(lambda x: postprocess_orgs(x))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [24]:
texts_df['GPE_lg'] = texts_df['ALL_NER_lg'].progress_apply(lambda x: postprocess_gpes(x))
texts_df['GPE_trf'] = texts_df['ALL_NER_trf'].progress_apply(lambda x: postprocess_gpes(x))
texts_df['GPE_flair'] = texts_df['ALL_NER_flair'].progress_apply(lambda x: postprocess_gpes(x))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [25]:
texts_df['LAW_lg'] = texts_df['ALL_NER_lg'].progress_apply(lambda x: postprocess_laws(x))
texts_df['LAW_trf'] = texts_df['ALL_NER_trf'].progress_apply(lambda x: postprocess_laws(x))
texts_df['LAW_flair'] = texts_df['ALL_NER_flair'].progress_apply(lambda x: postprocess_laws(x))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [26]:
texts_df['ORG_lg'] = texts_df['ORG_lg'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['ORG_trf'] = texts_df['ORG_trf'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['ORG_flair'] = texts_df['ORG_flair'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['GPE_lg'] = texts_df['GPE_lg'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['GPE_trf'] = texts_df['GPE_trf'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['GPE_flair'] = texts_df['GPE_flair'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['LAW_lg'] = texts_df['LAW_lg'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['LAW_trf'] = texts_df['LAW_trf'].progress_apply(lambda x: filter_by_frequency(x, 2))
texts_df['LAW_flair'] = texts_df['LAW_flair'].progress_apply(lambda x: filter_by_frequency(x, 2))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [27]:
type_org = (sorted(set([org[0] for orgs in texts_df['ORG_lg'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['ORG_trf'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['ORG_flair'] for org in orgs])))
type_gpe = (sorted(set([org[0] for orgs in texts_df['GPE_lg'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['GPE_trf'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['GPE_flair'] for org in orgs])))
type_law = (sorted(set([org[0] for orgs in texts_df['LAW_lg'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['LAW_trf'] for org in orgs])) 
+ sorted(set([org[0] for orgs in texts_df['LAW_flair'] for org in orgs])))
model_count_org = {}
for term in type_org:
    try:
        model_count_org[term] += 1
    except:
        model_count_org[term] = 1
model_count_gpe = {}
for term in type_gpe:
    try:
        model_count_gpe[term] += 1
    except:
        model_count_gpe[term] = 1
model_count_law = {}
for term in type_law:
    try:
        model_count_law[term] += 1
    except:
        model_count_law[term] = 1

In [28]:
texts_df['pre_ORG'] = texts_df.progress_apply(lambda x: filter_by_modelCount(x, 'ORG', model_count_org, 2), axis=1)
texts_df['pre_GPE'] = texts_df.progress_apply(lambda x: filter_by_modelCount(x, 'GPE', model_count_gpe, 2), axis=1)
texts_df['pre_LAW'] = texts_df.progress_apply(lambda x: filter_by_modelCount(x, 'LAW', model_count_law, 2), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [29]:
texts_df['pre_ORG']

0       [(PFS, PFS, to be explored in the PFS literatu...
1       [(Development, Development, Task CBO/ SOF EoI ...
2       [(GO Lab, GO Lab, to - social investment (GO L...
3       [(United Way, United Way, funds (). Currently,...
4       [(FIA, FIA, Interorganizational Linkages Incre...
                              ...                        
1947    [(PPP, PPP, is an inherent characteristic of P...
1948    [(Warner, Warner, evaluation of SIBs. Whereas ...
1949    [(Kreditanstalt Fr Wiederaufbau, Kreditanstalt...
1950    [(Department for International Development, De...
1951    [(RBF, RBF, some patients were enrolled in RBF...
Name: pre_ORG, Length: 1952, dtype: object

In [30]:
type_org = sorted(set([org[0] for orgs in texts_df['pre_ORG'].tolist() for org in orgs]))
type_gpe = sorted(set([gpe[0] for gpes in texts_df['pre_GPE'].tolist() for gpe in gpes]))
type_law = sorted(set([law[0] for laws in texts_df['pre_LAW'].tolist() for law in laws]))
type_all = type_org + type_gpe + type_law
texts_df['ORG'] = texts_df.progress_apply(lambda x: ent_from_oneEntType(x, 'ORG', type_all), axis=1)
texts_df['GPE'] = texts_df.progress_apply(lambda x: ent_from_oneEntType(x, 'GPE', type_all), axis=1)
texts_df['LAW'] = texts_df.progress_apply(lambda x: ent_from_oneEntType(x, 'LAW', type_all), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [31]:
texts_df['ORG']

0       [(PFS, PFS, to be explored in the PFS literatu...
1       [(Development, Development, Task CBO/ SOF EoI ...
2       [(GO Lab, GO Lab, to - social investment (GO L...
3       [(United Way, United Way, funds (). Currently,...
4       [(FIA, FIA, Interorganizational Linkages Incre...
                              ...                        
1947    [(PPP, PPP, is an inherent characteristic of P...
1948    [(Warner, Warner, evaluation of SIBs. Whereas ...
1949    [(Kreditanstalt Fr Wiederaufbau, Kreditanstalt...
1950    [(Department for International Development, De...
1951    [(PHC, PHC, TB patients were organized by PHC ...
Name: ORG, Length: 1952, dtype: object

In [32]:
print(len(set([org[0] for orgs in texts_df['ORG'] for org in orgs])))
print(len(set([org[0] for orgs in texts_df['GPE'] for org in orgs])))
print(len(set([org[0] for orgs in texts_df['LAW'] for org in orgs])))

13757
1944
125


In [33]:
texts_df.to_json("step4_output.json", orient="records", force_ascii=False)

# Step5: policy sector classification

In [34]:
import os
import re
import csv
import json
import spacy
import numpy as np
import pandas as pd
import tqdm
from qe import QueryExpansion
from preprocessor import read_stopwords, preprocessor
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [36]:
texts_df = pd.read_json("step4_output.json", orient="records")
texts_df.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,GPE_flair,LAW_lg,LAW_trf,LAW_flair,pre_ORG,pre_GPE,pre_LAW,ORG,GPE,LAW
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,"[[United States, United States, Safe Families ...",[],[],[],"[[PFS, PFS, to be explored in the PFS literatu...","[[America, America, Community Impact, Results ...",[],"[[PFS, PFS, to be explored in the PFS literatu...","[[America, America, Community Impact, Results ...",[]
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,"[[England, England, bonds (SIBs) in England, a...",[],[],[],"[[Development, Development, Task CBO/ SOF EoI ...","[[London, London, _of_ the_London_homelessness...",[],"[[Development, Development, Task CBO/ SOF EoI ...","[[Worcestershire, Worcestershire, benefit of t...",[]
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,"[[England, England, outcome based commissionin...",[],[],"[[VEAT, VEAT, Voluntary Ex Ante Transparency (...","[[GO Lab, GO Lab, to - social investment (GO L...","[[Midlands, Midlands, Award * Region East East...",[],"[[GO Lab, GO Lab, to - social investment (GO L...","[[Midlands, Midlands, Award * Region East East...",[]
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,"[[Chicago, Chicago, : Salt Lake City,Utah; Chi...",[],[],[],"[[United Way, United Way, funds (). Currently,...","[[Chicago, Chicago, Parent Centers preschool m...",[],"[[United Way, United Way, funds (). Currently,...","[[Utah, Utah, AND M. E. WARNER The Utah SIB ef...",[]
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,"[[Michigan, Michigan, , Managed Care Contracti...",[],[],[],"[[FIA, FIA, Interorganizational Linkages Incre...","[[Michigan, Michigan.3, care in Wayne County, ...",[],"[[FIA, FIA, Interorganizational Linkages Incre...","[[Michigan, Michigan.3, care in Wayne County, ...",[]


In [37]:
texts_df['title_abs'] = texts_df[['Title','Abstract']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract']), axis=1)
texts_df['full_txt'] = texts_df[['Title','Abstract','pdf_txt']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract'])+'\n'+str(row['pdf_txt']), axis=1)

In [38]:
from polyglot.text import Text, Word

language_dist = {}
langs = []
for idx, row in tqdm.tqdm(texts_df.iterrows()):
    if row['title_abs'] != None:
        sample = ''.join(x for x in row['title_abs'] if x.isprintable())
        text = Text(sample)
        langs.append(text.language.name)
#         print("Language Detected: Code={}, Name={}\n".format(text.language.code, text.language.name))
        try:
            language_dist[text.language.name] += 1
        except:
            language_dist[text.language.name] = 1
    else:
        langs.append("None")
texts_df['abs_lang'] = langs

language_dist = {}
langs = []
for idx, row in tqdm.tqdm(texts_df.iterrows()):
    if row['full_txt'] != None:
        sample = ''.join(x for x in row['full_txt'] if x.isprintable())
        text = Text(sample)
        langs.append(text.language.name)
#         print("Language Detected: Code={}, Name={}\n".format(text.language.code, text.language.name))
        try:
            language_dist[text.language.name] += 1
        except:
            language_dist[text.language.name] = 1
    else:
        langs.append("None")
texts_df['full_lang'] = langs

191it [00:00, 634.62it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
276it [00:00, 714.21it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
357it [00:00, 741.19it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
554it [00:00, 874.13it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
985it [00:01, 1030.21it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
1106it [00:01, 1081.62it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
1952it [00:01, 977.83it/s] 
392it [00:10, 40.75it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
612it [00:15, 43.10it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.
871it [00:21, 43.25it/s]WARNING:polyglot.detect.base:Detector is not able to detect the language reliably.


In [39]:
texts_df[texts_df['full_lang']!='English'].to_excel("non_english.xlsx")

In [40]:
from collections import Counter
Counter(texts_df['full_lang'])

Counter({'English': 1903,
         'Russian': 3,
         'Portuguese': 5,
         'Dutch': 2,
         'German': 4,
         'Finnish': 3,
         'Spanish': 4,
         'Japanese': 3,
         'Lithuanian': 3,
         'Italian': 4,
         'Korean': 3,
         'French': 8,
         'Maori': 1,
         'Croatian': 1,
         'Greek': 1,
         'Slovenian': 1,
         'Turkish': 1,
         'Chinese': 1,
         'Danish': 1})

In [41]:
stopwords = read_stopwords("./Data/stopwords.en.txt")
processor = preprocessor(stopwords)

In [42]:
texts_df['processed_title_abs'] = texts_df['title_abs'].progress_apply(lambda x: processor.preprocess(x))
texts_df['processed_full_txt'] = texts_df['full_txt'].progress_apply(lambda x: processor.preprocess(x))
texts_df['size_processed_title_abs'] = texts_df['processed_title_abs'].progress_apply(lambda x: len(x.split()))
texts_df['size_processed_full_txt'] = texts_df['processed_full_txt'].progress_apply(lambda x: len(x.split()))

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [43]:
# train = texts_df[(texts_df["abs_lang"] == "English") & (texts_df["size_processed_title_abs"] >= 3)]
# train = texts_df[(texts_df["full_lang"] == "English") & (texts_df["size_processed_full_txt"] >= 3)]
# train.head()

In [44]:
vobs = []
with open('./QDTM_input/vobs_FCDO_alltxt.txt', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        vobs.append(line.strip())

In [45]:
target = 7
feature = len(vobs)

def getWordTopicDistribtuion(path):
    zw = np.zeros((1000, feature))
    with open(path, 'r') as file:
        z = 0
        for line in file.readlines():
            w = 0
            for n in line.strip().split():
                zw[z,w] = n
                w += 1
            z += 1
    zw += 0.5
    pzw = zw/zw.sum(0)
    return pzw[:z]

def getTopicWordDistribtuion(path):
    zw = np.zeros((1000, feature))
    with open(path, 'r') as file:
        z = 0
        for line in file.readlines():
            w = 0
            for n in line.strip().split():
                zw[z,w] = n
                w += 1
            z += 1
    zw += 0.5
    pzw = zw/zw.sum(1)[:,np.newaxis]
    return pzw[:z]

In [46]:
data_samples = texts_df['processed_full_txt'].to_list()
vectorizer = CountVectorizer(vocabulary = vobs) #for abstract + title
vectors = vectorizer.transform(data_samples).toarray()

In [47]:
path = './QDTM_results/QDTM-FCDO-alltxt mannual keywords T20_nzw.txt'
pzw = getWordTopicDistribtuion(path)
pwz = getTopicWordDistribtuion(path)
X = (vectors + 0.01) @ pzw.T

In [48]:
for i, topic in enumerate(pwz):
    print("topic " + str(i))
    for i in topic.argsort()[::-1][:10]:
        print(vobs[i], end=" ")
    print()

topic 0
hospital patient medical treatment health doctor psychiatric disease physician medication 
topic 1
school education high teaching student teacher college graduate educational kindergarten 
topic 2
housing residential homelessness outcome sii client accommodation rough report australia 
topic 3
program government pfs service project social success state pay trial 
topic 4
work working job employment people programme participant young provider unemployment 
topic 5
care child service support practice agency family home data outcome 
topic 6
quality management resource finance conservation economy sustainability cost project energy 
topic 7
outcome bond impact dib evaluation service dibs cost provider stakeholder 
topic 8
facility effect group analysis study variable table data service difference 
topic 9
health pbf facility service rbf study district system intervention financing 
topic 10
research respondent policy organisation process study interview issue sector worker 
topic 

In [49]:
labels = ["health", "education", "homelessness", "criminal", "employment", "child", "agriculture"]
for i in range(target):
    print(labels[i], (X[:,:target].argsort(1)[:,::-1][:,0] == i).sum())

health 419
education 89
homelessness 17
criminal 139
employment 117
child 854
agriculture 317


In [50]:
labels = ["health", "education", "homelessness", "criminal", "employment", "child", "agriculture"]
pre = [labels[i] for i in list(X[:,:target].argsort(1)[:,::-1][:,0])]
# class_table = {"id": texts_df['id'].to_list(), "pdfs":texts_df['pdf_file'].to_list(), "Title":texts_df['Title'].to_list(), "Abstract":texts_df['Abstract'].to_list(), "class":pre}
normalized_dist = X[:,:target]/X[:,:target].sum(1)[:,np.newaxis]
for i in range(target):
    texts_df[labels[i]+"_prob"] = list(normalized_dist[:,i])

In [51]:
texts_df["health"] = texts_df["health_prob"].progress_apply(lambda x: x if x >= 0.24 else None)
texts_df["education"] = texts_df["education_prob"].progress_apply(lambda x: x if x >= 0.2 else None)
texts_df["homelessness"] = texts_df["homelessness_prob"].progress_apply(lambda x: x if x >= 0.12 else None)
texts_df["criminal"] = texts_df["criminal_prob"].progress_apply(lambda x: x if x >= 0.29 else None)
texts_df["employment"] = texts_df["employment_prob"].progress_apply(lambda x: x if x >= 0.2 else None)
texts_df["child"] = texts_df["child_prob"].progress_apply(lambda x: x if x >= 0.41 else None)
texts_df["agriculture"] = texts_df["agriculture_prob"].progress_apply(lambda x: x if x >= 0.28 else None)

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

In [52]:
texts_df.to_json("step5_output.json", orient="records", force_ascii=False)

# Step6: search Countries from articles

In [53]:
countries_file = 'Data/Countries/country-codes.csv'
worldbank_file = 'Data/Countries/world_bank_class.csv'
texts_df = pd.read_json("step5_output.json", orient="records")
texts_df['txt'] = texts_df[['Title','Abstract','pdf_txt']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract'])+'\n'+str(row['pdf_txt']), axis=1)
texts_df.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,employment_prob,child_prob,agriculture_prob,health,education,homelessness,criminal,employment,child,agriculture
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,0.106774,0.451090,0.087874,NaN,NaN,NaN,NaN,NaN,0.451090,NaN
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,0.211451,0.270156,0.142589,NaN,NaN,NaN,NaN,0.211451,NaN,NaN
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,0.222086,0.270308,0.142761,NaN,NaN,NaN,NaN,0.222086,NaN,NaN
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,0.116288,0.199748,0.135383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,0.115480,0.553074,0.078477,NaN,NaN,NaN,NaN,NaN,0.553074,NaN


### Read country list

In [54]:
country_codes_df = pd.read_csv(countries_file)
country_codes_dict_df = country_codes_df[['official_name_en',
                                          'UNTERM English Formal','UNTERM English Short']]
country_codes_dict_df

,official_name_en,UNTERM English Formal,UNTERM English Short
0,NaN,NaN,NaN
1,Afghanistan,the Islamic Republic of Afghanistan,Afghanistan
2,Albania,the Republic of Albania,Albania
3,Algeria,the People's Democratic Republic of Algeria,Algeria
4,American Samoa,NaN,NaN
...,...,...,...
245,Western Sahara,NaN,NaN
246,Yemen,the Republic of Yemen,Yemen
247,Zambia,the Republic of Zambia,Zambia
248,Zimbabwe,the Republic of Zimbabwe,Zimbabwe


In [55]:
countries_list = country_codes_dict_df['official_name_en'].tolist()+ \
country_codes_dict_df['UNTERM English Formal'].tolist()+ \
country_codes_dict_df['UNTERM English Short'].tolist()

countries_list = list(set(countries_list)-{np.nan})

countries_list_new = []
for countr in countries_list:
    temp_countr = countr
    if temp_countr[-6:] == ' (the)':
        temp_countr = countr[:-6]
    if temp_countr[-3:] == '  *':
        temp_countr = countr[:-3]
    if temp_countr[-9:] == ' (the)  *':
        temp_countr = countr[:-9]
    if re.search('\(',temp_countr):
        temp_countr = re.sub(' \(.*\)','',temp_countr)
    if re.search('\*',temp_countr):
        temp_countr = re.sub('\*','',temp_countr)
    countries_list_new.append(temp_countr)
    
countries_list = countries_list_new   

countries_list = list(set(countries_list))

In [56]:
texts_df

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,employment_prob,child_prob,agriculture_prob,health,education,homelessness,criminal,employment,child,agriculture
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,0.106774,0.451090,0.087874,NaN,NaN,NaN,NaN,NaN,0.451090,NaN
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,0.211451,0.270156,0.142589,NaN,NaN,NaN,NaN,0.211451,NaN,NaN
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,0.222086,0.270308,0.142761,NaN,NaN,NaN,NaN,0.222086,NaN,NaN
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,0.116288,0.199748,0.135383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,0.115480,0.553074,0.078477,NaN,NaN,NaN,NaN,NaN,0.553074,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,Innovation in Australian publicly operated pub...,"Saeed, Ali Mohammed; Duffield, Colin; Hui, Felix",It is challenging to identify or evaluate inno...,2019.0,None,INFRASTRUCTURE ASSET MANAGEMENT,6,3,166-177,None,...,0.139167,0.224924,0.206923,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948,Social impact bonds: The evolution of research...,"Broccardo, Eleonora; Mazzuca, Maria; Frigotto,...",This study reviews all the academic contributi...,2020.0,None,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,27,3,1316-1332,None,...,0.149144,0.211362,0.186465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949,Quality of Care in Performance-Based Financing...,"Gergen, Jessica; Josephson, Erik; Coe, Martha;...",Objective: To describe how quality of care is ...,2017.0,None,GLOBAL HEALTH-SCIENCE AND PRACTICE,5,1,90-107,None,...,0.090138,0.257997,0.276611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950,Payment by Results in Development Aid: All Tha...,"Clist, Paul","Payment by Results (PbR), where aid is disburs...",2016.0,None,WORLD BANK RESEARCH OBSERVER,31,2,290-313,None,...,0.172697,0.202530,0.182969,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Read World Bank classification list

In [57]:
worldbank_df = pd.read_csv(worldbank_file,header=None)
countries_list = list(set(countries_list + worldbank_df[1].tolist()))
worldbank_list = list(zip(worldbank_df[0].tolist(),worldbank_df[1].tolist()))

### Search Countries in Articles

In [58]:
def txt_el_search(txt, el_list):
    found_els = []
    for el in el_list:
        temp = re.findall(fr"[^a-zA-Z]({str(el)})[^a-zA-Z]", txt, re.IGNORECASE)
        if len(temp) > 1:
            found_els.append(str(el))
    return found_els

In [59]:
texts_df['txt_countries'] = texts_df['txt'].progress_apply(lambda x: txt_el_search(x,countries_list))

  0%|          | 0/1952 [00:00<?, ?it/s]

### WorldBank classification of mentioned countries

In [60]:
def txt_el_class_search(countr_list, class_list):
    found_els = []
    for cl in class_list:
        countr_list_low = [countr.lower() for countr in countr_list]
        if cl[1].lower() in countr_list_low:
            found_els.append(cl[0])
    found_els = sorted(list(set(found_els)))
    return found_els

In [61]:
texts_df['txt_countries_class'] = texts_df['txt_countries'].progress_apply(lambda x: txt_el_class_search(x,worldbank_list))

  0%|          | 0/1952 [00:00<?, ?it/s]

In [62]:
texts_df.to_json("step6_output.json", orient="records", force_ascii=False)

# Step7: extract financial mechanisms

In [63]:
texts_df = pd.read_json("step6_output.json", orient="records")
# texts_df2 = pd.read_json("step6_output.json", orient="records")[['id','pdf_txt']]
# texts_df = pd.merge(texts_df, texts_df2, on=['id'],  how='left')
texts_df['txt'] = texts_df[['Title','Abstract','pdf_txt']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract'])+'\n'+str(row['pdf_txt']), axis=1)
financial_mechanisms = pd.read_excel('Data/financial mechanisms/financial mechanisms.xlsx', engine='openpyxl')
financial_mechanisms = financial_mechanisms.fillna('')
financial_mechanisms.head()
full2Abb = {re.sub("\s+", " ", re.sub(r"\xa0", " ", term.lower().strip())): financial_mechanisms['Abbreviation'][index] for index, term in enumerate(financial_mechanisms['Term'])}

def txt_financial_search(txt, el_list):
    found_els = []
    for el in terms:
        temp_el = re.sub("\*", "[a-zA-Z]*", el)
        temp = re.findall(fr"{temp_el}", txt, re.IGNORECASE)
        if len(temp) > 1:
            found_els.append(el)
    return set(found_els)

def get_top_financial_mechanisms(row):
    entities = row["financial_mechanisms"]
    txt = row['txt']
    id_ = row['Covidence #']
    output = []
    for ent in entities:
        temp_org = re.sub("\*", "[a-zA-Z]*", ent)
#         context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*{temp_org}(?: ?[^ ]+){{0,5}}", txt, re.IGNORECASE)
        context = re.findall(fr"\s.{{,30}}{temp_org}.{{,30}}\s", txt, re.IGNORECASE)
        context = [{"content":sample, "id": id_, "entity": re.findall(temp_org, sample, re.IGNORECASE)[0]} for sample in context]

        if len(full2Abb[ent]) > 0:
            context2 = re.findall(fr"\s.{{,30}}[^a-zA-Z]{full2Abb[ent]}s?[^a-zA-Z].{{,30}}\s", txt, re.IGNORECASE)
            context2 = [{"content":sample, "id": id_, "entity": full2Abb[ent]} for sample in context2]
            context.extend(context2)

        if len(context) < 2:
            print(ent)
            continue

        output.append({"name": ent, "count": len(context), "context":context})
        
    return [ent for ent in sorted(output,key=lambda x:-x["count"])]

terms = list(set(re.sub("\s+", " ", re.sub(r"\xa0", " ", term.lower().strip())) for term in financial_mechanisms['Term'].tolist()))
texts_df['financial_mechanisms'] = texts_df['txt'].progress_apply(lambda x: txt_financial_search(x, terms))
texts_df['top_financial_mechanisms'] = texts_df.progress_apply(lambda x: get_top_financial_mechanisms(x), axis=1)
texts_df.to_json("step7_output.json", orient="records", force_ascii=False)

  0%|          | 0/1952 [00:00<?, ?it/s]

  0%|          | 0/1952 [00:00<?, ?it/s]

payment-by-result*
outcome*-base* commissioning
payment-by-result*
payment-by-result*
contract* for health*
social impact invest*
performance-base* contract*
development impact bond*
public-private partnership*
result*-base* financ*
performance-base* financ*
performance-base* contract*
performance-base* incentive*


# Step8: extract sdgs

In [65]:
texts_df = pd.read_json("step7_output.json", orient="records")
sdg_terms = ["No poverty",
            "Zero hunger",
            "Good health and wellbeing",
            "Quality education",
            "Gender equality",
            "Clean water and sanitation",
            "Affordable and clean energy",
            "Work and economic growth",
            "Industry, innovation and infrastructure",
            "Reduced inequalities",
            "Sustainable cities and communities",
            "Responsible consumption and production",
            "Climate action",
            "Life below water",
            "Life on land",
            "Peace, justice and strong institutions",
            "Partnerships for the goals",
            "Sustainable Development Goal",
            "Sustainable Development Goals",
            "Sdg",
            "Sdgs"]

def txt_sdg_search(txt, orgs_list):
    temp_txt = txt
    found_orgs = []
    for org in orgs_list:
        mentioned = re.findall(fr"{org}", temp_txt, re.IGNORECASE)

        if mentioned:
            found_orgs.append((str(org), len(mentioned)))
#             found_orgs.append(str(org))
    return found_orgs

def get_top_sdgs(row):
    entities = row["mentioend_sdgs"]
    txt = row['txt']
    id_ = row['Covidence #']
    output = []
    context_by_ent = {}
    for tup in entities:
        ent = tup[0]
        context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*[^a-zA-Z]{ent}[^a-zA-Z](?: ?[^ ]+){{0,5}}", txt, re.IGNORECASE)
        context = [{"content":sample, "id": id_, "entity": ent} for sample in context]
        
        if ent.lower() in ["sustainable development goal", "sustainable development goals", "sdg", "sdgs"]:
            temp = "Sustainable Development Goals"
        else:
            temp = ent
            
        try:
            context_by_ent[temp].extend(context)
        except:
            context_by_ent[temp] = []
            context_by_ent[temp].extend(context)
    
    for ent in context_by_ent:
        if len(context_by_ent[ent]) < 2:
            print(ent)
            continue
        output.append({"name": ent, "count": len(context_by_ent[ent]), "context":context_by_ent[ent]})
        
    return [ent for ent in sorted(output,key=lambda x:-x["count"])]

txt_sdgs = []
elements = []
for idx, row in tqdm.tqdm(texts_df[['txt']].iterrows()):
    temp = txt_sdg_search(row["txt"], sdg_terms)
    txt_sdgs.append(temp)
    elements += temp
texts_df['mentioend_sdgs'] = txt_sdgs
print(len(set(elements)))
texts_df['top_sgds'] = texts_df.progress_apply(lambda x: get_top_sdgs(x), axis=1)
texts_df.to_json("step8_output.json", orient="records", force_ascii=False)

1952it [01:04, 30.13it/s]

117


  0%|          | 0/1952 [00:00<?, ?it/s]

Quality education
Gender equality
Quality education
Reduced inequalities
Climate action
Life below water
Life on land
Sustainable Development Goals
Sustainable Development Goals
Zero hunger
Clean water and sanitation
Climate action
No poverty
Climate action
Life on land
Quality education
Quality education
Gender equality
Sustainable Development Goals
Gender equality
Gender equality
Quality education
Quality education
Gender equality
Gender equality
Gender equality
Affordable and clean energy
Work and economic growth
Quality education
Climate action
No poverty
Partnerships for the goals
Sustainable Development Goals
Sustainable Development Goals
Reduced inequalities
Climate action
Sustainable Development Goals
Quality education
No poverty
Zero hunger
Work and economic growth
Industry, innovation and infrastructure
Peace, justice and strong institutions
Quality education
Gender equality
Quality education
Clean water and sanitation
Gender equality
Climate action
Gender equality
Sustainabl

# Step9: Assgin country code and extract context around countries and projects

In [66]:
texts_df = pd.read_json("step8_output.json", orient="records")
texts_df['txt'] = texts_df[['Title','Abstract','pdf_txt']].apply(
    lambda row: str(row['Title'])+'\n'+str(row['Abstract'])+'\n'+str(row['pdf_txt']), axis=1)
texts_df[["ORG","LAW","GPE","txt_countries","txt_countries_class"]].head()

,ORG,LAW,GPE,txt_countries,txt_countries_class
0,"[[PFS, PFS, to be explored in the PFS literatu...",[],"[[America, America, Community Impact, Results ...","[United States, United Kingdom]",[HIC]
1,"[[Development, Development, Task CBO/ SOF EoI ...",[],"[[Worcestershire, Worcestershire, benefit of t...",[],[]
2,"[[GO Lab, GO Lab, to - social investment (GO L...",[],"[[Midlands, Midlands, Award * Region East East...",[],[]
3,"[[United Way, United Way, funds (). Currently,...",[],"[[Utah, Utah, AND M. E. WARNER The Utah SIB ef...",[United States],[HIC]
4,"[[FIA, FIA, Interorganizational Linkages Incre...",[],"[[Michigan, Michigan.3, care in Wayne County, ...",[],[]


In [67]:
country_codes_df = pd.read_csv('Data/Countries/country-codes.csv')
country_codes_dict_df = country_codes_df[['ISO3166-1-Alpha-2' ,'ISO3166-1-Alpha-3','official_name_en',
                                          'UNTERM English Formal','UNTERM English Short', 'CLDR display name']]
country_codes_dict_df = country_codes_dict_df.dropna(subset=["ISO3166-1-Alpha-3"])
country_codes_dict_df

,ISO3166-1-Alpha-2,ISO3166-1-Alpha-3,official_name_en,UNTERM English Formal,UNTERM English Short,CLDR display name
0,TW,TWN,NaN,NaN,NaN,Taiwan
1,AF,AFG,Afghanistan,the Islamic Republic of Afghanistan,Afghanistan,Afghanistan
2,AL,ALB,Albania,the Republic of Albania,Albania,Albania
3,DZ,DZA,Algeria,the People's Democratic Republic of Algeria,Algeria,Algeria
4,AS,ASM,American Samoa,NaN,NaN,American Samoa
...,...,...,...,...,...,...
245,EH,ESH,Western Sahara,NaN,NaN,Western Sahara
246,YE,YEM,Yemen,the Republic of Yemen,Yemen,Yemen
247,ZM,ZMB,Zambia,the Republic of Zambia,Zambia,Zambia
248,ZW,ZWE,Zimbabwe,the Republic of Zimbabwe,Zimbabwe,Zimbabwe


In [68]:
def get_gpe(row, ent_type):
    countries = row[ent_type]
    txt = row['txt']
    id_ = row['Covidence #']
    context_by_ISO3 = {}
    output = []
    for gpe in countries:
        if ent_type == "txt_countries":
            temp = gpe
        else:
            temp = gpe[0]
            
        if (temp == "USA") or (temp == "US") or (temp == "America") or (temp == "United States"):
            temp = 'United States of America'
        elif (temp == "UK") or (temp == "the United Kingdom’s") or (temp == "United Kingdom") \
         or (temp == "the United Kingdom"):
            temp = 'United Kingdom of Great Britain and Northern Ireland'
            
        if ent_type == "txt_countries":
            temp_gpe = None
            if (temp in country_codes_dict_df['official_name_en'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['official_name_en'] == temp]['ISO3166-1-Alpha-3'].item()
            elif (temp in country_codes_dict_df['UNTERM English Formal'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Formal'] == temp]['ISO3166-1-Alpha-3'].item()
            elif (temp in country_codes_dict_df['UNTERM English Short'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Short'] == temp]['ISO3166-1-Alpha-3'].item()
            elif (temp in country_codes_dict_df['CLDR display name'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['CLDR display name'] == temp]['ISO3166-1-Alpha-3'].item()
            if temp_gpe == None:
                continue
#             context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*{gpe}(?: ?[^ ]+){{0,5}}", txt)
            context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*[^a-zA-Z]{gpe}[^a-zA-Z](?: ?[^ ]+){{0,5}}", txt, re.IGNORECASE)
            context = [{"content":sample, "id": id_, "entity": gpe} for sample in context]
    
        else:
            temp_gpe = temp
            if (temp in country_codes_dict_df['official_name_en'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['official_name_en'] == temp]['CLDR display name'].item()
            elif (temp in country_codes_dict_df['UNTERM English Formal'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Formal'] == temp]['CLDR display name'].item()
            elif (temp in country_codes_dict_df['UNTERM English Short'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Short'] == temp]['CLDR display name'].item()
            elif (temp in country_codes_dict_df['ISO3166-1-Alpha-3'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['ISO3166-1-Alpha-3'] == temp]['CLDR display name'].item()
            elif (temp in country_codes_dict_df['ISO3166-1-Alpha-2'].tolist()):
                temp_gpe = country_codes_dict_df[country_codes_dict_df['ISO3166-1-Alpha-2'] == temp]['CLDR display name'].item()
#             context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*{gpe[1]}(?: ?[^ ]+){{0,5}}", txt)

            context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*[^a-zA-Z]{re.escape(gpe[1])}[^a-zA-Z](?: ?[^ ]+){{0,5}}", txt, re.IGNORECASE)
            context = [{"content":sample, "id": id_, "entity": gpe[1]} for sample in context]

            
        try:
            context_by_ISO3[temp_gpe].extend(context)
        except:
            context_by_ISO3[temp_gpe] = []
            context_by_ISO3[temp_gpe].extend(context)
    
    for country in context_by_ISO3:
        if len(context_by_ISO3[country]) < 2:
            print(country)
            continue
        output.append({"name": country, "count": len(context_by_ISO3[country]), "context":context_by_ISO3[country]})

    return [ent for ent in sorted(output,key=lambda x:-x["count"])]

In [69]:
texts_df['top_countries'] = texts_df.progress_apply(lambda x:get_gpe(x, "txt_countries"), axis=1)
# texts_df['top_GPE'] = texts_df.progress_apply(lambda x:get_gpe(x, "GPE"), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

AUS
HTI
HUN
JEY
BGR
JEY
NER
BRA
PHL
BGD
IRL
BEL
HUN
ARG
GRC
NGA
DNK
SGP
ZWE
CHN
UGA
AUS
MYS
SGP
GHA
ARG
GEO
JEY
UGA
THA
ISR


In [70]:
import ast

def get_top_projects(row):
    entities = row["mentioned_projects"]
    txt = row['txt']
    id_ = row['Covidence #']
    output = []
    for ent in entities:
        temp_org = ent
        temp_org = re.sub("[^a-zA-Z0-9 ]", "", temp_org.strip())
        temp_org = "[^a-zA-Z0-9 ]*".join([c for c in temp_org])
        context = re.findall(fr"(?:[^ ]+ ){{0,5}}[^ ]*[^a-zA-Z]{temp_org}[^a-zA-Z](?: ?[^ ]+){{0,5}}", txt, re.IGNORECASE)
#         context = re.findall(fr"\s.{{,30}}[^a-zA-Z]{re.escape(temp_org)}[^a-zA-Z].{{,30}}\s", txt, re.IGNORECASE)
        if len(context) < 2:
            print(ent)
            continue
        context = [{"content":sample, "id": id_ } for sample in context]
        output.append({"name": ent, "count": len(context), "context":context})
        
    return [ent for ent in sorted(output,key=lambda x:-x["count"])]

In [71]:
texts_df['top_Project'] = texts_df.progress_apply(lambda x:get_top_projects(x), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

Prevista
Cameroon Cataract Bond
Pause


In [72]:
texts_df.to_json("step9_output.json", orient="records", force_ascii=False)

# Step10: Merge duplicate entities

In [73]:
texts_df = pd.read_json("step9_output.json", orient="records")
texts_df.head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,...,child,agriculture,txt_countries,txt_countries_class,financial_mechanisms,top_financial_mechanisms,mentioend_sdgs,top_sgds,top_countries,top_Project
0,Innovation In Child Welfare Through Pay For Su...,"Jon Klinepeter, Former Pay For Success Directo...",The United States foster care system is in cri...,2019.0,None,None,None,None,None,None,...,0.451090,NaN,"[United States, United Kingdom]",[HIC],"[performance-base* contract*, pay for success,...","[{'name': 'pay for success', 'count': 108, 'co...",[],[],"[{'name': 'USA', 'count': 6, 'context': [{'con...",[]
1,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Tim Fox; Stanworth, Neil",The CBO Fund aims to support the development o...,2016.0,None,None,None,None,None,None,...,NaN,NaN,[],[],"[impact bond*, outcome-base* payment*, social ...","[{'name': 'social impact bond*', 'count': 612,...",[],[],[],[]
2,Commissioning Better Outcomes Fund Evaluation ...,"James Ronicle, Neil Stanworth; Hickman, Edward",This report is an updated review of the Commis...,2019.0,None,None,None,None,None,None,...,NaN,NaN,[],[],"[impact bond*, outcome-base* payment*, develop...","[{'name': 'social impact bond*', 'count': 957,...",[],[],[],"[{'name': 'West London Zone', 'count': 9, 'con..."
3,The RazorS Edge: Social Impact Bonds And The F...,"Warner, Allison E. Tse & Mildred E.","In a growing number of U.S. cities, social imp...",2018.0,None,None,None,None,None,None,...,NaN,NaN,[United States],[HIC],"[social impact bond*, impact bond*, pay for su...","[{'name': 'social impact bond*', 'count': 160,...",[],[],"[{'name': 'USA', 'count': 10, 'context': [{'co...",[{'name': 'South Carolina Nurse-Family Partner...
4,"Nonprofit Adaptation to Performance-Based, Man...","McBeath, Bowen; Meezan, William",While performance-based & managed care contrac...,2006.0,None,Administration in Social Work,30.0,2,39-70,61383844; 200701417,...,0.553074,NaN,[],[],[performance-base* contract*],"[{'name': 'performance-base* contract*', 'coun...",[],[],[],[]


In [74]:
org_list = set([org[0] for orgs in texts_df["ORG"] for org in orgs])
org_dict = {}
for org in tqdm.tqdm(org_list):
    if len(org.split()) == 1:
        for org2 in org_list:
            if org != org2 and re.search(fr"\b{org}\b", org2):
                try:
                    org_dict[org].append(org2)
                except:
                    org_dict[org] = []
                    org_dict[org].append(org2)
my_custom_terms = []
for org in org_dict:
    if len(org_dict[org]) >=10:
        my_custom_terms.append(org.lower())
my_custom_terms.extend(["church", "children", "estate", "author", "art", "operation"])

100%|██████████| 13757/13757 [02:58<00:00, 77.27it/s] 


In [75]:
sdg_terms = set([sdg[0].lower() for sdgs in texts_df["mentioend_sdgs"] for sdg in sdgs])
financial_mechanism_terms = set([context['entity'].lower() for fms in texts_df["top_financial_mechanisms"] for fm in fms for context in fm['context']])
project_terms = set([project['name'].lower().strip() for projects in texts_df["top_Project"] for project in projects])
part_exclude_terms = sdg_terms | financial_mechanism_terms | project_terms

my_custom_terms = set(my_custom_terms)
Golab_custom_terms = set("State, NGO, NGOs, Success, Medicare, Medicaid, Elsevier, Elsevier Ltd, Healthcare, Education, Board, \
Health Affairs, Eds, Lancet, Time, Group, Social Impact Bond, Educate Girls DIB, Urban, Social, Program, Programme, \
Project, Coalition, Participant, Bank, A&E, Gustafsson-Wright, House, Committee, Elsevier BV, \
Health Policy and Planning, Commission, Commonwealth, Trust, Cooperative, Cooperatives, Health Policy, NVivo, \
Quality and Outcomes Framework, VCSE, Hospital, Target, Guardian, Public Private Partnership, \
Public Private Partnerships, Wang, Research, Federal, Center, Centre, Social Work, ANOVA, Provider, Office, \
Partnerships, Management, Partnership, Social Security, Families, Grade, P4P, SPV, ProQuest LLC, ProQuest, MDG, \
Sage, Sage Publications, Routledge, New Public Management, NPM, Social Science & Medicine, Facility, Ibid, \
Medical Care Research and Review, Health Policy and Planning, Housing, School, Global, Evaluation, Energy, \
Public, Community, Employment, Higher Education, Services, Service, Children, College, Cohort, Development, \
Affordable Housing, Learning, National, System, Nursing, Biodiesel, Innovation, Social Innovation, Public Housing, \
Pay, Study, Regent Park, VfM, Arts, Pilot Agency, ESCO, SME, Women, Projects, Review, S Texas, \
National Troubled Families Initiative, SROI, Focus, Bond, Clean Cities, Practice".lower().split(", "))
full_exclude_terms = my_custom_terms | Golab_custom_terms
full_exclude_terms

{'a&e',
 'academy',
 'access',
 'accountability',
 'affordable housing',
 'agency',
 'agriculture',
 'alliance',
 'analysis',
 'anova',
 'art',
 'arts',
 'assembly',
 'associates',
 'association',
 'author',
 'authority',
 'bank',
 'biodiesel',
 'board',
 'bond',
 'budget',
 'bureau',
 'cabinet',
 'capital',
 'care',
 'ccg',
 'census',
 'center',
 'centers',
 'central',
 'centre',
 'centres',
 'children',
 'church',
 'clean',
 'clean cities',
 'clinic',
 'coalition',
 'cohort',
 'collaborative',
 'college',
 'commerce',
 'commission',
 'committee',
 'commons',
 'commonwealth',
 'communities',
 'community',
 'company',
 'consortium',
 'control',
 'cooperative',
 'cooperatives',
 'corporation',
 'corrections',
 'council',
 'court',
 'department',
 'development',
 'division',
 'economics',
 'eds',
 'educate girls dib',
 'education',
 'elsevier',
 'elsevier bv',
 'elsevier ltd',
 'employment',
 'energy',
 'environmental',
 'esco',
 'estate',
 'european',
 'evaluation',
 'exchange',
 'execu

In [76]:
import textdistance

country_codes_df = pd.read_csv('Data/Countries/country-codes.csv')
country_codes_dict_df = country_codes_df[['ISO3166-1-Alpha-2' ,'ISO3166-1-Alpha-3','official_name_en',
                                          'UNTERM English Formal','UNTERM English Short', 'CLDR display name']]
country_codes_dict_df = country_codes_dict_df.dropna(subset=["ISO3166-1-Alpha-3"])
country_codes_dict_df

def merge_duplicate_gpe(gpe_list):
    mergeDict = {}
    for gpe in gpe_list:
        temp = gpe
        if (temp == "USA") or (temp == "US") or (temp == "America") or (temp == "United States"):
            temp = 'United States of America'
        elif (temp == "UK") or (temp == "the United Kingdom’s") or (temp == "United Kingdom") \
         or (temp == "the United Kingdom"):
            temp = 'United Kingdom of Great Britain and Northern Ireland'
            
        temp_gpe = temp
        if (temp in country_codes_dict_df['official_name_en'].tolist()):
            temp_gpe = country_codes_dict_df[country_codes_dict_df['official_name_en'] == temp]['CLDR display name'].item()
        elif (temp in country_codes_dict_df['UNTERM English Formal'].tolist()):
            temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Formal'] == temp]['CLDR display name'].item()
        elif (temp in country_codes_dict_df['UNTERM English Short'].tolist()):
            temp_gpe = country_codes_dict_df[country_codes_dict_df['UNTERM English Short'] == temp]['CLDR display name'].item()
        elif (temp in country_codes_dict_df['CLDR display name'].tolist()):
            temp_gpe = country_codes_dict_df[country_codes_dict_df['CLDR display name'] == temp]['CLDR display name'].item()
        
        mergeDict[gpe] = temp_gpe
    return mergeDict

def merge_duplicate_law(law_list):
    mergeDict = {}
    for law in law_list:
        temp_law = law
        if law == "General Appropriations Acts":
            temp_law = "General Appropriations Act"
        if law == "Social Security Amendments":
            temp_law = "Social Security Amendments Act"
        mergeDict[law] = temp_law
    return mergeDict       
#         mergeDict[gpe] = temp_gpe
#     return mergeDict

def merge_duplicate_org(org_list):
    mergeDict = {}
    for org in tqdm.tqdm(org_list):
        temp = re.sub("U\.K\.?", "UK ", org)
        temp = re.sub("U\.S\.?", "US ", temp)
        temp = re.sub("\s+", " ", temp).strip()
        country_start = re.search(r"^([0-9a-zA-Z]+)", temp, re.IGNORECASE)
        if country_start and (country_start.group(1) in country_codes_dict_df['official_name_en'].tolist()
                or country_start.group(1) in country_codes_dict_df['UNTERM English Formal'].tolist()
                or country_start.group(1) in country_codes_dict_df['UNTERM English Short'].tolist()
                or country_start.group(1) in country_codes_dict_df['CLDR display name'].tolist()):
            temp = re.sub(r"^[0-9a-zA-Z]+-?", "", temp, re.IGNORECASE).strip()

        country_end = re.search(r"(of((?!of)[ 0-9a-zA-Z])*?and ([0-9a-zA-Z]*)$)", temp, re.IGNORECASE)
        country_end2 = re.search(r"(of)? ([0-9a-zA-Z]*)$", temp, re.IGNORECASE)
        country_end3 = re.search("New Mexico$", temp, re.IGNORECASE)
        country_end4 = re.search("the People’s Republic of [0-9a-zA-Z]*$", temp, re.IGNORECASE)
        if country_end and (country_end.group(3) in country_codes_dict_df['official_name_en'].tolist()
                or country_end.group(3) in country_codes_dict_df['UNTERM English Formal'].tolist()
                or country_end.group(3) in country_codes_dict_df['UNTERM English Short'].tolist()
                or country_end.group(3) in country_codes_dict_df['CLDR display name'].tolist()):
            temp = re.sub(r"(of((?!of|in)[ 0-9a-zA-Z])*?and ([0-9a-zA-Z]*)$)", "", temp, re.IGNORECASE).strip()
        elif country_end2 and (country_end2.group(2).strip() in country_codes_dict_df['official_name_en'].tolist()
                or country_end2.group(2).strip() in country_codes_dict_df['UNTERM English Formal'].tolist()
                or country_end2.group(2).strip() in country_codes_dict_df['UNTERM English Short'].tolist()
                or country_end2.group(2).strip() in country_codes_dict_df['CLDR display name'].tolist()):
            if country_end3:
                temp = re.sub("(of|in)? ?(The)? ?New Mexico$", "", temp, re.IGNORECASE).strip()
            elif country_end4:
                temp = re.sub("(of|in)? ?(the)? ?People’s Republic of [0-9a-zA-Z]*$", "", temp, re.IGNORECASE).strip()
            else:
                temp = re.sub("(of|in)? ?(The)? [0-9a-zA-Z]*$", "", temp, re.IGNORECASE).strip()
        temp = " ".join([word.capitalize() for word in temp.split()])
        mergeDict[org] = temp
    
    cand_orgs = list(org_list) + list(mergeDict.values())
    for org in set(cand_orgs):
        for k in mergeDict:
            if org + "s" == mergeDict[k]:
                mergeDict[k] = org
                break
    
    return mergeDict
gpe_list = set([gpe[0] for gpes in texts_df["GPE"] for gpe in gpes if gpe[0].lower() not in full_exclude_terms])
gpe_merge_dict = merge_duplicate_gpe(gpe_list)     

law_list = set([law[0] for laws in texts_df["LAW"] for law in laws if law[0].lower() not in full_exclude_terms])
law_merge_dict = merge_duplicate_law(law_list)   

org_list = set([org[0] for orgs in texts_df["ORG"] for org in orgs if org[0].lower() not in full_exclude_terms])
org_merge_dict = merge_duplicate_org(org_list)   

100%|██████████| 13536/13536 [00:01<00:00, 7673.11it/s]


## Get context around entites

In [77]:
def get_entity_context(row, ent_type, part_exclude_terms, full_exclude_terms, merge_dict):
    context_by_ent = {}
    entities = row[ent_type]
    txt = row['txt']
    id_ = row['Covidence #']
    output = []
    for ent in entities:
        temp_org = ent.copy()
        if temp_org[0].lower() in full_exclude_terms or temp_org[1].lower() in full_exclude_terms:
            continue
            
        stop = False
        for term in part_exclude_terms:
            if term.lower() in temp_org[0].lower() or term.lower() in temp_org[1].lower():
                stop = True
                break
        if stop:
            continue
       
        temp = re.sub("U\.K\.?", "UK ", temp_org[0])
        temp = re.sub("U\.S\.?", "US ", temp)
        temp = re.sub("\s+", " ", temp).strip()
        if temp_org[0].lower() == merge_dict[temp_org[0]].lower() + "s":
            sample = {"content":temp_org[2], "id": id_, "entity": temp_org[1], "sub_entity": merge_dict[temp_org[0]]} 
        else:
            if merge_dict[temp_org[0]].lower() == temp.lower():
                sample = {"content":temp_org[2], "id": id_, "entity": temp_org[1], "sub_entity": merge_dict[temp_org[0]]}
            else:
                sample = {"content":temp_org[2], "id": id_, "entity": temp_org[1], "sub_entity": temp}
        try:
            context_by_ent[merge_dict[temp_org[0]]].append(sample)
        except:
            context_by_ent[merge_dict[temp_org[0]]] = []
            context_by_ent[merge_dict[temp_org[0]]].append(sample)
            
    for ent in context_by_ent:
        if len(context_by_ent[ent]) < 2 or ent in full_exclude_terms:
            print(ent)
            continue
        output.append({"name": ent, "count": len(context_by_ent[ent]), "context":context_by_ent[ent]})
    return [ent for ent in sorted(output,key=lambda x:-x["count"])]
                           
texts_df['top_GPE'] = texts_df.progress_apply(lambda x:get_entity_context(x, "GPE", part_exclude_terms, full_exclude_terms, gpe_merge_dict), axis=1)
texts_df['top_LAW'] = texts_df.progress_apply(lambda x:get_entity_context(x, "LAW", part_exclude_terms, full_exclude_terms, law_merge_dict), axis=1)
texts_df['top_ORG'] = texts_df.progress_apply(lambda x:get_entity_context(x, "ORG", part_exclude_terms, full_exclude_terms, org_merge_dict), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

Burrowes
Lyon
Borough of Ipswich
Suffolk Coastal
Porto
Southbank
Alaska
portugal
Tallahassee
Italy
UK
Blatchford
ethiopia
Kenedy County
Thousand Oaks
Melbourne
Mildon
Panahpur
Cagliari
New South Wales
Vantaa
CANADA
Ireland
South Korea
Panahpur
New York
UK
England
CHL
Germany
Kansas City
Scotland
St. Louis
Newark
Eugene
Richmond
Cleveland
Kansas City
Dallas
Tallahassee
Culley
Burrowes
U.S.A
Austria
New York
Brighton
El Salvador
Ethiopia
Bonn
Burrowes
Warwickshire
RIKERS ISLAND
PETERBOROUGH
Nigeria
SENEGAL
Lietuvai
El Kef
Tanzania
Druskininkuose
UK
India
Venice
UK
Washington, DC
San Francisco
RIKERS ISLAND
PETERBOROUGH
Macedonia
Moldova
New York
Rikers Island
HKSAR
Niamey
Zinder
Sheffield
US
Rikers Island
Boise
Mukono
New York
Shandong
US
East London
US
Norway
Akabas
Albany
Madison
Los Angeles
Slovenia
Israel
Greece
Chile
York
Saarbrucken
Bernalillo County
Shrestha
Sydney
Hyattsville
Mongolia
England
Sri Lanka
Wisconsin
San Francisco
HABINEZA
MARYLAND
Anderson
Bronx
Ningxia Province
Minn

  0%|          | 0/1952 [00:00<?, ?it/s]

Affordable Care Act
BPHS Guidelines
BPHS Guidelines


  0%|          | 0/1952 [00:00<?, ?it/s]

Hmp Peterborough
Health And Social Care
Atq
One Service
Ojeu
Atq Consultants
Health And Social Care
Vcses
Broadway
Ghk Consulting
Special Purpose Vehicle
Research Centre For Entrepreneurship & Business Innovation
Girls’ Education Challenge
Web Of Science
Newcastle Ways To Wellness Social Prescribing Service
Manchester Tfco
Nvq
Ukbi
Border Force
Contact Centre
Ag2r La Mondiale
Mckinsey
Bodleian Libraries
Grant And Contract Management Unit
Phd
Finance Department
Llc
Bureau Of Indian Affairs
Southern Regional Education Board
Office Of The Superintendent
Third Sector Capital Partners
Eca
Bbc News
Heinz Foundation
Committee For Economic Development
Invest In Kids Working Group
Early Childhood Partnerships
Student Achievement
Llc
School Of Public Health
Farmers Market
National League Of Major League Baseball
Ohrpp
Policylink
Bps
Statistics
International Finance Corporation
Department Of Water Affairs
Pir
Public Health
Sha
Salford Royal Foundation Trust
Nchs
Mcap
Uncitral
Ark Schools
Social I

Nic
National Quality Forum
Premier
Premier Inc
World Health Organization Regional Office For Africa
Esy
Eopy
Pir
Police National Computer
Cochrane Library
Chinese Cochrane Center
World Bank’s Role
House Of Representatives
Committee On Appropriations
Global Health
United States General Accounting Office
Development Assistance Committee
Global Health
Dnp
National Commission
Ecological Economics
Ipts
Jrc
Government
Nsw Hospital
Danish Ministry Of Foreign Affairs
International Development Research Centre
Idrc
Ministry Of Foreign Affairs
Rti
Korea International Cooperation Agency
Health Research Educational Trust
Svp
Advisory Board Company
Group Health Mutual Insurance Company
Office Of The Actuary
Critical Access Hospitals
Lake Superior Quality Innovation Network
Governance Board Of Trustees
Nhe
Ols
Congress
College Of Public Health
Qbe Insurance
Prospero
California Water Board
Environmental Protection Agency
California Department Of Toxic Substances Control
Cambridge Core
Health Maintenan

## Convert Abbs to their full names

In [78]:
import nltk
from nltk.corpus import stopwords
stopwords_list = set(stopwords.words('english'))
    
def abb2full(row, part_exclude_terms, full_exclude_terms, org_merge_dict):
    orgs = row["top_ORG"]
    abb2full = {}
    for org in orgs:
        for context in org['context']:
            try:
                abbs_name = re.search(fr"{org['name']}([^a-zA-Z]\((?: ?[^ ]+)\))", context["content"], re.IGNORECASE)
                full_name = " ".join([word.capitalize() for word in org['name'].split()])
                cap_name = "".join([word.capitalize()[0] for word in org['name'].split()])
                if abbs_name:
                    abbs_name = abbs_name.group(1).strip(" ()")
                    temp = "[a-zA-Z]*".join([c for c in abbs_name])
                    if re.search(fr"{temp}", cap_name, re.IGNORECASE):
                        try:
                            abb2full[abbs_name.capitalize()].add(full_name)
                        except:
                            abb2full[abbs_name.capitalize()] = set()
                            abb2full[abbs_name.capitalize()].add(full_name)

                temp = re.sub("\s*[-–]\s+", "", context["content"])
                temp = re.sub("[-–]", " ", temp)
                temp = re.sub("\s+", " ", temp)
                full_name = re.search(fr"((?:[^ ]+ ){{0,{len(org['name'])}}}[^ ]*)[^a-zA-Z]{org['name']}", temp, re.IGNORECASE).group(1)
                full_name = " ".join([word.capitalize() for word in full_name.split()]).strip()
                preceed = "".join([word[0] for word in full_name.split()])
                if org['name'].lower() == preceed.lower():
                    if len(full_name) != 0:
                        try:
                            abb2full[org['name'].capitalize()].add(full_name)
                        except:
                            abb2full[org['name'].capitalize()] = set()
                            abb2full[org['name'].capitalize()].add(full_name)
            except:
                continue  
                
    results = []
    context_by_ent = {}
    for org in orgs:
#         if len(org['name'].split()) > 1:
#             org['name'] = " ".join([word.capitalize() for word in org['name'].split()])
        temp_org = org.copy()
        if temp_org['name'] in abb2full and len(abb2full[temp_org['name']]) == 1 and list(abb2full[temp_org['name']])[0].lower() not in full_exclude_terms:
            full_name = list(abb2full[temp_org['name']])[0]
            stop = False
            for term in part_exclude_terms:
                if term.lower() in full_name.lower():
                    stop = True
                    break
            if re.search("(for a$|for$|of$|Fund$|Survey$|Reports?$|Programmes?$|Programs?$|SIBs?$|Chapters?$| NA$|Schemes?$|Plans?$|Reform$|Bonds?$|Systems?$|Services?$|Outcomes?$|Acts?$)", full_name, re.IGNORECASE):
                stop = True
            if stop:
                continue
                
            if full_name in org_merge_dict:
                if full_name.lower() == org_merge_dict[full_name].lower() + "s":
                    for context in temp_org['context']:
                        context['sub_entity'] = org_merge_dict[full_name]
                    temp_org['name'] = org_merge_dict[full_name]
                else:
                    if org_merge_dict[full_name].lower() == full_name.lower():
                        for context in temp_org['context']:
                            context['sub_entity'] = org_merge_dict[full_name]
                        temp_org['name'] = org_merge_dict[full_name]
                    else:
                        for context in temp_org['context']:
                            context['sub_entity'] = full_name
                        temp_org['name'] = org_merge_dict[full_name]
            else:
                for context in temp_org['context']:
                    context['sub_entity'] = full_name
                temp_org['name'] = full_name
        
        try:
            context_by_ent[temp_org['name']].extend(temp_org['context'])
        except:
            context_by_ent[temp_org['name']] = []
            context_by_ent[temp_org['name']].extend(temp_org['context'])

    results = []
    for ent in context_by_ent:  
        results.append({"name": ent, "count": len(context_by_ent[ent]), "context":context_by_ent[ent]})
    return results

texts_df['top_ORG'] = texts_df.progress_apply(lambda x:abb2full(x, part_exclude_terms, full_exclude_terms, org_merge_dict), axis=1)

  0%|          | 0/1952 [00:00<?, ?it/s]

<ipython-input-78-6595ff595541>:46: FutureWarning: Possible nested set at position 41
  if re.search(fr"{temp}", cap_name, re.IGNORECASE):
<ipython-input-78-6595ff595541>:46: FutureWarning: Possible nested set at position 1
  if re.search(fr"{temp}", cap_name, re.IGNORECASE):


In [ ]:
texts_df.to_json("step10_output.json", orient="records", force_ascii=False)

# Step11: attach project urls

In [34]:
texts_df = pd.read_json("step10_output.json", orient="records")
texts_df.head()

,id,Covidence #,pdf_file,Title,Abstract,Authors,Published Year,DOI,mentioned_projects,policy_sector_health,...,policy_sector_child,policy_sector_environment,financial_mechanisms,top_financial_mechanisms,top_sgds,top_countries,top_Project,top_GPE,top_LAW,top_ORG
0,0,#17157,#17157 - JonKlinepeter 2019.pdf,Innovation In Child Welfare Through Pay For Su...,The United States foster care system is in cri...,"Jon Klinepeter, Former Pay For Success Directo...",2019.0,None,[],NaN,...,0.451090,NaN,"[performance-base* contract*, pay for success,...","[{'name': 'pay for success', 'count': 108, 'co...",[],"[{'name': 'USA', 'count': 6, 'context': [{'con...",[],"[{'name': 'Illinois', 'count': 9, 'context': [...",[],"[{'name': 'Safe Families', 'count': 84, 'conte..."
1,1,#17224,#17224 - JamesRonicle 2016.pdf,Commissioning Better Outcomes Fund Evaluation ...,The CBO Fund aims to support the development o...,"James Ronicle, Tim Fox; Stanworth, Neil",2016.0,None,[],NaN,...,NaN,NaN,"[impact bond*, outcome-base* payment*, social ...","[{'name': 'social impact bond*', 'count': 612,...",[],[],[],"[{'name': 'England', 'count': 9, 'context': [{...",[],"[{'name': 'Ways To Wellness', 'count': 43, 'co..."
2,2,#17223,#17223 - JamesRonicle 2019.pdf,Commissioning Better Outcomes Fund Evaluation ...,This report is an updated review of the Commis...,"James Ronicle, Neil Stanworth; Hickman, Edward",2019.0,None,[West London Zone],NaN,...,NaN,NaN,"[impact bond*, outcome-base* payment*, develop...","[{'name': 'social impact bond*', 'count': 957,...",[],[],"[{'name': 'West London Zone', 'count': 9, 'con...","[{'name': 'England', 'count': 7, 'context': [{...",[],"[{'name': 'Hct', 'count': 21, 'context': [{'co..."
3,3,#17284,#17284 - Warner 2018.pdf,The RazorS Edge: Social Impact Bonds And The F...,"In a growing number of U.S. cities, social imp...","Warner, Allison E. Tse & Mildred E.",2018.0,None,[South Carolina Nurse-Family Partnership Pay f...,NaN,...,NaN,NaN,"[social impact bond*, impact bond*, pay for su...","[{'name': 'social impact bond*', 'count': 160,...",[],"[{'name': 'USA', 'count': 10, 'context': [{'co...",[{'name': 'South Carolina Nurse-Family Partner...,"[{'name': 'Utah', 'count': 44, 'context': [{'c...",[],"[{'name': 'Ece', 'count': 29, 'context': [{'co..."
4,4,#4965,McBeath-2006-Nonprofit Adaptation to Performan...,"Nonprofit Adaptation to Performance-Based, Man...",While performance-based & managed care contrac...,"McBeath, Bowen; Meezan, William",2006.0,http://dx.doi.org/10.1300/J147v30n02_05,[],NaN,...,0.553074,NaN,[performance-base* contract*],"[{'name': 'performance-base* contract*', 'coun...",[],[],[],"[{'name': 'Wayne County', 'count': 11, 'contex...",[],"[{'name': 'Fia', 'count': 43, 'context': [{'co..."


In [35]:
projects = pd.read_csv('./Data/Indigo/Impact Bond Dataset with URLs.csv')
projects.head()
project2url = {}
for i, project in projects.iterrows():
    project2url[str(project['Impact Bond Name - (Value)']).strip()] = project['URL']
    project2url[str(project['Alternative Names - (Value)']).strip()] = project['URL']
for i, row in texts_df.iterrows():
    for project in row['top_Project']:
        if project['name'].strip() in project2url:
            project['URL'] = project2url[project['name'].strip()]
        else:
            project['URL'] = ""

In [79]:
texts_df = texts_df[["id","Covidence #", "pdf_file","Title","Abstract","Authors","Published Year", "DOI", "mentioned_projects","health","education",
          "homelessness","criminal","employment","child","agriculture", "financial_mechanisms", "top_financial_mechanisms", "top_sgds", "top_countries", "top_Project", 
            "top_GPE", "top_LAW", "top_ORG"]]
texts_df = texts_df.rename(columns={
                          'health':'policy_sector_health',
                          'education':'policy_sector_education',
                          'homelessness':'policy_sector_homelessness',
                          'criminal':'policy_sector_criminal',
                          'employment':'policy_sector_employment',
                          'child':'policy_sector_child',
                          'agriculture':'policy_sector_environment'}
                          )

In [36]:
texts_df.to_excel("info_by_articlesV6.xlsx")
texts_df.to_json("info_by_articlesV6.json", orient="records", force_ascii=False)